In [11]:
#!/usr/bin/env python3
"""
Component 1: Markov-Switching Regime Detection
===============================================
Uses Hamilton (1989) Markov-switching regression via statsmodels.
Now includes pre-2020 robustness check to assess COVID-era distortion.
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import statsmodels.api as sm

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]
FED_CHAIRS = [
    ("Burns","1970-02","1978-01"),("Miller","1978-03","1979-08"),
    ("Volcker","1979-08","1987-08"),("Greenspan","1987-08","2006-01"),
    ("Bernanke","2006-02","2014-01"),("Yellen","2014-02","2018-02"),
    ("Powell","2018-02","2024-12"),
]

def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()

    # CBO NAIRU (quarterly -> monthly)
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")

    # Michigan Survey expected inflation
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}),
            how="left")
    else:
        merged["expected_inflation"] = np.nan

    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["delta_pi"] = df["inflation"].diff()
    df["delta_u"] = df["unemployment"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation","unemp_gap","capacity_gap","fed_funds","term_spread","fin_conditions"]:
        for lag in [1,3,6,12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    for h in [1,3,6,12]:
        df[f"inflation_{h}m_ahead"] = df["inflation"].shift(-h)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation","L12_inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


# ── Markov-Switching Model ─────────────────────────────────────────
def fit_markov_switching(df, n_regimes=2, label=""):
    prefix = f"[{label}] " if label else ""
    print(f"\n{'='*70}")
    print(f"{prefix}MARKOV-SWITCHING REGIME MODEL ({n_regimes} regimes, n={len(df)})")
    print(f"{'='*70}")

    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())

    mod = sm.tsa.MarkovRegression(
        endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True,
    )

    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf:
                best_llf = res.llf; best_res = res
        except Exception:
            continue

    if best_res is None:
        print("  Model failed to converge.")
        return None

    res = best_res
    print(f"\n  Log-likelihood: {res.llf:.1f}  |  AIC: {res.aic:.1f}  |  BIC: {res.bic:.1f}")

    tm = res.regime_transition
    tm_avg = tm.mean(axis=2) if tm.ndim == 3 else tm
    print(f"\n  Transition matrix (columns = from, rows = to):")
    header = "  " + " "*12 + "".join([f"  From R{j}" for j in range(n_regimes)])
    print(header)
    for i in range(n_regimes):
        row = f"  To R{i}:     "
        for j in range(n_regimes):
            row += f"  {tm_avg[i,j]:7.4f}"
        print(row)

    durations = res.expected_durations
    print(f"\n  Expected durations:")
    for r in range(n_regimes):
        print(f"    Regime {r}: {durations[r]:.1f} months ({durations[r]/12:.1f} years)")

    smoothed = res.smoothed_marginal_probabilities
    regime_df = df.copy()

    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0

    sorted_regimes = sorted(regime_means, key=regime_means.get)
    for new_r in range(n_regimes):
        old_r = sorted_regimes[new_r]
        regime_df[f"p_regime_{new_r}"] = smoothed[old_r].values

    regime_df["regime"] = np.argmax(
        np.column_stack([regime_df[f"p_regime_{r}"].values for r in range(n_regimes)]),
        axis=1)

    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    if n_regimes == 2:
        labels = ["Low", "High"]
    elif n_regimes == 3:
        labels = ["Low", "Medium", "High"]
    else:
        labels = [f"Regime {r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}

    print(f"\n  {'Regime':<30s}  {'N':>5s}  {'pi':>7s}  {'u':>7s}  {'ff':>7s}  {'spread':>7s}  {'E[dur]':>8s}")
    print(f"  {'-'*80}")
    for r in range(n_regimes):
        sub = regime_df[regime_df["regime"] == r]
        dur = durations[sorted_regimes[r]]
        print(f"  {regime_names[r]:<30s}  {len(sub):>5d}  {sub['inflation'].mean():>7.2f}  "
              f"{sub['unemployment'].mean():>7.2f}  {sub['fed_funds'].mean():>7.2f}  "
              f"{sub['term_spread'].mean():>7.2f}  {dur:>7.1f}m")

    print(f"\n{res.summary()}")
    return res, regime_df, regime_names, n_regimes, sorted_regimes, durations


def select_n_regimes(df, max_k=4):
    print(f"\n  Model selection by BIC:")
    results = {}
    for k in range(2, max_k + 1):
        endog = df["inflation"].copy()
        exog = sm.add_constant(df[["unemployment"]].copy())
        mod = sm.tsa.MarkovRegression(endog, k_regimes=k, exog=exog,
            switching_variance=True, switching_exog=True)
        best_res, best_llf = None, -np.inf
        for attempt in range(5):
            try:
                res = mod.fit(search_reps=20, random_state=SEED+attempt, disp=False, maxiter=500)
                if res.llf > best_llf:
                    best_llf = res.llf; best_res = res
            except:
                continue
        if best_res:
            results[k] = best_res
            print(f"    k={k}: LL={best_res.llf:>8.1f}  AIC={best_res.aic:>8.1f}  BIC={best_res.bic:>8.1f}")
    if not results:
        return 2
    best_k = min(results, key=lambda k: results[k].bic)
    print(f"    -> Selected: k={best_k}")
    return best_k


# ── Pre-2020 robustness check ──────────────────────────────────────
def robustness_pre2020(df):
    """Re-estimate MS model ending sample at 2019:12 to check COVID distortion."""
    print(f"\n{'='*70}")
    print("ROBUSTNESS: PRE-2020 SAMPLE (excluding COVID-era)")
    print(f"{'='*70}")

    df_pre = df.loc[:"2019-12"].copy()
    print(f"  Pre-2020 sample: {len(df_pre)} obs ({df_pre.index.min():%Y-%m} to {df_pre.index.max():%Y-%m})")

    result_pre = fit_markov_switching(df_pre, n_regimes=2, label="Pre-2020")
    if result_pre is None:
        print("  Pre-2020 model failed.")
        return None

    _, rdf_pre, rn_pre, _, _, dur_pre = result_pre

    # Compare to full sample
    print(f"\n  --- Comparison: Pre-2020 vs Full sample ---")
    return result_pre


# ── Visualization ──────────────────────────────────────────────────
def plot_regimes(df, regime_df, regime_names, n_regimes, sorted_regimes, durations,
                 output_path, suffix=""):
    regime_colors = [C5, C1, C4, C2, "#94a3b8"][:n_regimes]
    sfx = f"_{suffix}" if suffix else ""

    # Fig 1: Smoothed probabilities
    n_panels = n_regimes + 1
    fig, axes = plt.subplots(n_panels, 1, figsize=(16, 2.5 + 2.2*n_regimes), sharex=True)
    if n_panels == 1: axes = [axes]
    fig.suptitle(f"Markov-Switching Inflation Regimes ({n_regimes} states): Smoothed Probabilities",
                 fontweight="bold", fontsize=14, y=0.995)

    ax = axes[0]; shade_recessions(ax)
    for r in range(n_regimes):
        mask = regime_df["regime"] == r
        ax.scatter(regime_df.index[mask], regime_df["inflation"][mask],
                   c=regime_colors[r], s=6, alpha=0.7, label=regime_names[r], zorder=2)
    ax.axhline(2, color="grey", ls="--", lw=1, alpha=0.5)
    ax.set_ylabel("Inflation (%)"); ax.set_title("Inflation by most likely regime", fontweight="bold")
    ax.legend(frameon=True, fontsize=7, markerscale=2, loc="upper right")

    for r in range(n_regimes):
        ax = axes[r + 1]; shade_recessions(ax)
        ax.fill_between(regime_df.index, regime_df[f"p_regime_{r}"], 0,
                        alpha=0.4, color=regime_colors[r])
        ax.plot(regime_df.index, regime_df[f"p_regime_{r}"], lw=1, color=regime_colors[r])
        ax.axhline(0.5, color="grey", ls="--", lw=0.8, alpha=0.5)
        ax.set_ylabel("P(regime)")
        dur = durations[sorted_regimes[r]]
        ax.set_title(f"P({regime_names[r]}) | E[duration]: {dur:.0f} months",
                     fontweight="bold", fontsize=10)
        ax.set_ylim(-0.05, 1.05)

    axes[-1].set_xlabel("Year")
    fig.tight_layout()
    path1 = f"{output_path}/ms_regime_probabilities{sfx}.png"
    fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path1}")

    # Fig 2: Regime timeline + macro context
    fig, axes = plt.subplots(4, 1, figsize=(16, 12), sharex=True,
                              gridspec_kw={"height_ratios": [1, 2, 2, 2]})
    fig.suptitle("Inflation Regimes in Macroeconomic Context",
                 fontweight="bold", fontsize=14, y=0.995)

    ax = axes[0]
    for r in range(n_regimes):
        mask = regime_df["regime"] == r
        for d in regime_df.index[mask]:
            ax.axvspan(d, d + pd.DateOffset(months=1), color=regime_colors[r], alpha=0.8)
    for i, (ch, s, e) in enumerate(FED_CHAIRS):
        mid = pd.Timestamp(s) + (pd.Timestamp(e)-pd.Timestamp(s))/2
        if regime_df.index[0] <= mid <= regime_df.index[-1]:
            ax.text(mid, 0.5, ch, ha="center", va="center", fontsize=8,
                    fontstyle="italic", alpha=0.7)
    ax.set_yticks([])
    ax.set_title("Regime assignment with Fed chairs", fontweight="bold")
    legend_elements = [Patch(facecolor=regime_colors[r], alpha=0.8, label=regime_names[r])
                       for r in range(n_regimes)]
    ax.legend(handles=legend_elements, frameon=True, fontsize=7, loc="upper left",
              ncol=min(n_regimes,3))

    ax = axes[1]; shade_recessions(ax)
    ax.plot(regime_df.index, regime_df["inflation"], lw=1.5, color="#374151")
    ax.axhline(2, color=C2, ls="--", lw=1, alpha=0.5, label="2% target")
    ax.set_ylabel("Inflation (%)"); ax.set_title("YoY Inflation", fontweight="bold")
    ax.legend(frameon=False, fontsize=8)

    ax = axes[2]; shade_recessions(ax)
    ax.plot(regime_df.index, regime_df["fed_funds"], lw=1.5, color=C1, label="Fed funds")
    ax.plot(regime_df.index, regime_df["treasury_10y"], lw=1.2, color=C2, alpha=0.6, label="10Y Treasury")
    ax.set_ylabel("Rate (%)"); ax.set_title("Interest rates", fontweight="bold")
    ax.legend(frameon=False, fontsize=8)

    ax = axes[3]; shade_recessions(ax)
    ax.plot(regime_df.index, regime_df["unemployment"], lw=1.5, color=C3, label="Unemployment")
    ax.plot(regime_df.index, regime_df["nrou"], lw=1.2, color=C4, ls="--",
            alpha=0.7, label="CBO NAIRU")
    ax.set_ylabel("Rate (%)"); ax.set_xlabel("Year")
    ax.set_title("Unemployment rate vs CBO natural rate", fontweight="bold")
    ax.legend(frameon=False, fontsize=8)

    fig.tight_layout()
    path2 = f"{output_path}/ms_regime_context{sfx}.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # Fig 3: Distributions by regime
    fig, axes = plt.subplots(2, 3, figsize=(16, 8))
    fig.suptitle("Regime Characteristics", fontweight="bold", fontsize=14, y=1.01)

    for ax, (var, label) in zip(axes.flat, [
        ("inflation","Inflation (%)"), ("unemployment","Unemployment (%)"),
        ("fed_funds","Fed funds (%)"), ("real_rate","Real rate (%)"),
        ("term_spread","Term spread (pp)"), ("unemp_gap","Unemployment gap (pp, CBO NAIRU)")]):

        data_list = [regime_df.loc[regime_df["regime"]==r, var].dropna().values
                     for r in range(n_regimes)]
        bp = ax.boxplot(data_list, patch_artist=True, widths=0.6,
                        labels=[f"R{r}" for r in range(n_regimes)],
                        medianprops=dict(color="black", lw=1.5))
        for r, patch in enumerate(bp["boxes"]):
            patch.set_facecolor(regime_colors[r]); patch.set_alpha(0.6)
        ax.set_title(label, fontweight="bold", fontsize=10)
        ax.axhline(0, color="grey", lw=0.5, alpha=0.3)

    fig.tight_layout()
    path3 = f"{output_path}/ms_regime_distributions{sfx}.png"
    fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path3}")

    return [path1, path2, path3]


# ── Data overview plot ─────────────────────────────────────────────
def plot_data_overview(df, output_path):
    fig, axes = plt.subplots(3, 2, figsize=(16, 10), sharex=True)
    fig.suptitle("US Economic Indicators, 1971-2024", fontweight="bold", fontsize=14, y=1.0)

    panels = [
        ("inflation", "CPI Inflation (%)", C1),
        ("unemployment", "Unemployment (%)", C3),
        ("fed_funds", "Fed Funds Rate (%)", C2),
        ("capacity_util", "Capacity Utilisation (%)", C4),
        ("treasury_10y", "10-Year Treasury (%)", C5),
        ("fin_conditions", "Financial Conditions (NFCI)", "#6b7280"),
    ]
    for ax, (var, title, color) in zip(axes.flat, panels):
        shade_recessions(ax)
        ax.plot(df.index, df[var], lw=1.2, color=color)
        ax.set_title(title, fontweight="bold", fontsize=10)

    # Add NAIRU to unemployment panel
    axes[0, 1].plot(df.index, df["nrou"], lw=1, color=C4, ls="--", alpha=0.6, label="CBO NAIRU")
    axes[0, 1].legend(frameon=False, fontsize=7)

    axes[-1, 0].set_xlabel("Year"); axes[-1, 1].set_xlabel("Year")
    fig.tight_layout()
    path = f"{output_path}/data_overview.png"
    fig.savefig(path, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path}")
    return path


# ── Main ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    # Data overview
    plot_data_overview(df, OUTPUT_PATH)

    # Model selection
    best_k = select_n_regimes(df, max_k=4)

    # Main model
    result = fit_markov_switching(df, n_regimes=best_k, label="Full sample")
    all_paths = []
    if result is not None:
        res, regime_df, regime_names, n_regimes, sorted_regimes, durations = result
        paths = plot_regimes(df, regime_df, regime_names, n_regimes,
                            sorted_regimes, durations, OUTPUT_PATH, suffix=f"k{best_k}")
        all_paths.extend(paths)

    # Always show 2-regime for comparison
    if best_k != 2:
        print("\n\n--- 2-regime baseline for comparison ---")
        result2 = fit_markov_switching(df, n_regimes=2, label="2-regime baseline")
        if result2 is not None:
            res2, rdf2, rn2, n2, sr2, dur2 = result2
            paths2 = plot_regimes(df, rdf2, rn2, n2, sr2, dur2, OUTPUT_PATH, suffix="k2")
            all_paths.extend(paths2)

    # Pre-2020 robustness
    result_pre = robustness_pre2020(df)
    if result_pre is not None:
        res_pre, rdf_pre, rn_pre, n_pre, sr_pre, dur_pre = result_pre
        paths_pre = plot_regimes(df.loc[:"2019-12"], rdf_pre, rn_pre, n_pre,
                                 sr_pre, dur_pre, OUTPUT_PATH, suffix="pre2020")
        all_paths.extend(paths_pre)

    print(f"\nAll outputs: {all_paths}")
    print("Done.")

Merged: 857 obs (1954-07 to 2025-12)
  Saved: /Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs/data_overview.png

  Model selection by BIC:
    k=2: LL= -1483.3  AIC=  2986.6  BIC=  3033.8
    k=3: LL= -1184.3  AIC=  2404.5  BIC=  2489.6
    k=4: LL= -1002.0  AIC=  2060.0  BIC=  2192.3
    -> Selected: k=4

[Full sample] MARKOV-SWITCHING REGIME MODEL (4 regimes, n=833)

  Log-likelihood: -1002.0  |  AIC: 2060.0  |  BIC: 2192.3

  Transition matrix (columns = from, rows = to):
                From R0  From R1  From R2  From R3
  To R0:        0.9404   0.0353   0.0335   0.0000
  To R1:        0.0325   0.9647   0.0096   0.0000
  To R2:        0.0272   0.0000   0.9410   0.0361
  To R3:       -0.0000   0.0000   0.0159   0.9639

  Expected durations:
    Regime 0: 16.8 months (1.4 years)
    Regime 1: 28.3 months (2.4 years)
    Regime 2: 17.0 months (1.4 years)
    Regime 3: 27.7 months (2.3 years)

  Regime                              N       pi        u       ff   spread    E

In [12]:
#!/usr/bin/env python3
"""
Component 2: Regime-Conditional Taylor Rules
=============================================
Key changes from original:
- CBO NAIRU replaces hardcoded 5% unemployment gap
- Expected inflation (MICH / adaptive proxy) specification alongside realized
- HAC standard errors via block bootstrap
- Pre-2020 robustness check
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import statsmodels.api as sm
from scipy.optimize import least_squares
from scipy import stats

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]
FED_CHAIRS = [
    ("Burns","1970-02","1978-01"),("Miller","1978-03","1979-08"),
    ("Volcker","1979-08","1987-08"),("Greenspan","1987-08","2006-01"),
    ("Bernanke","2006-02","2014-01"),("Yellen","2014-02","2018-02"),
    ("Powell","2018-02","2024-12"),
]

def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else:
        merged["expected_inflation"] = np.nan
    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["delta_pi"] = df["inflation"].diff()
    df["delta_u"] = df["unemployment"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation","unemp_gap","capacity_gap","fed_funds","term_spread","fin_conditions"]:
        for lag in [1,3,6,12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    for h in [1,3,6,12]:
        df[f"inflation_{h}m_ahead"] = df["inflation"].shift(-h)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation","L12_inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


# ── Get regimes from Markov-switching ──────────────────────────────
def get_regimes(df, n_regimes=2):
    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())
    mod = sm.tsa.MarkovRegression(endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True)
    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf: best_llf = res.llf; best_res = res
        except: continue
    if best_res is None:
        return df, {0: "Full sample"}, 1
    smoothed = best_res.smoothed_marginal_probabilities
    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0
    sorted_regimes = sorted(regime_means, key=regime_means.get)
    rdf = df.copy()
    for new_r in range(n_regimes):
        rdf[f"p_regime_{new_r}"] = smoothed[sorted_regimes[new_r]].values
    rdf["regime"] = np.argmax(
        np.column_stack([rdf[f"p_regime_{r}"].values for r in range(n_regimes)]), axis=1)
    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    labels = ["Low","High"] if n_regimes==2 else [f"R{r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}
    return rdf, regime_names, n_regimes


# ── Taylor Rule NLS (CBO NAIRU) ───────────────────────────────────
def taylor_resid(params, y, pi, u_gap, ff_lag):
    """Residuals for structural Taylor rule using unemployment gap directly."""
    rho, r_star, a_pi, a_u = params
    target = r_star + pi + a_pi * (pi - 2.0) + a_u * u_gap
    return y - (rho * ff_lag + (1 - rho) * target)


def taylor_resid_expected(params, y, pi_e, u_gap, ff_lag):
    """Residuals using expected inflation instead of realized."""
    rho, r_star, a_pi, a_u = params
    target = r_star + pi_e + a_pi * (pi_e - 2.0) + a_u * u_gap
    return y - (rho * ff_lag + (1 - rho) * target)


def fit_taylor_nls(sub, label="", use_expected=False):
    """Fit structural Taylor rule via NLS. Uses CBO NAIRU for unemployment gap."""
    y = sub["fed_funds"].values
    u_gap = sub["unemp_gap"].values  # already unemployment - NAIRU
    ff_lag = sub["ff_lag1"].values

    if use_expected:
        pi = sub["pi_expected"].values
        valid = ~np.isnan(pi)
        if valid.sum() < 30:
            return None
        y, pi, u_gap, ff_lag = y[valid], pi[valid], u_gap[valid], ff_lag[valid]
        resid_fn = taylor_resid_expected
    else:
        pi = sub["inflation"].values
        resid_fn = taylor_resid

    res = least_squares(resid_fn, [0.85, 2.0, 0.5, 0.5],
        args=(y, pi, u_gap, ff_lag),
        bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]))

    rho, rstar, a_pi, a_u = res.x
    n = len(y)
    sigma2 = np.sum(res.fun**2) / (n - 4)
    J = res.jac
    try:
        cov = sigma2 * np.linalg.inv(J.T @ J)
        se = np.sqrt(np.diag(cov))
    except:
        se = [np.nan]*4

    target = rstar + pi + a_pi * (pi - 2.0) + a_u * u_gap
    prescribed = np.maximum(target, 0.0)
    ssr = np.sum(res.fun**2)

    return {
        "label": label, "n": n, "use_expected": use_expected,
        "rho": rho, "rstar": rstar, "a_pi": a_pi, "a_u": a_u,
        "se": se, "ssr": ssr, "sigma2": sigma2,
        "prescribed": prescribed, "actual": y,
        "taylor_principle": 1 + a_pi,
    }


# ── HAC standard errors via block bootstrap ───────────────────────
def bootstrap_taylor_se(sub, n_boot=500, block_size=12, use_expected=False):
    """Block bootstrap for HAC-robust standard errors on NLS Taylor rule."""
    n = len(sub)
    boot_params = []

    for b in range(n_boot):
        # Block bootstrap indices
        n_blocks = int(np.ceil(n / block_size))
        block_starts = np.random.randint(0, n - block_size + 1, size=n_blocks)
        indices = np.concatenate([np.arange(s, s + block_size) for s in block_starts])[:n]

        boot_sub = sub.iloc[indices].copy()
        y = boot_sub["fed_funds"].values
        u_gap = boot_sub["unemp_gap"].values
        ff_lag = boot_sub["ff_lag1"].values

        if use_expected:
            pi = boot_sub["pi_expected"].values
            valid = ~np.isnan(pi)
            if valid.sum() < 30:
                continue
            y, pi, u_gap, ff_lag = y[valid], pi[valid], u_gap[valid], ff_lag[valid]
            resid_fn = taylor_resid_expected
        else:
            pi = boot_sub["inflation"].values
            resid_fn = taylor_resid

        try:
            res = least_squares(resid_fn, [0.85, 2.0, 0.5, 0.5],
                args=(y, pi, u_gap, ff_lag),
                bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]),
                max_nfev=300)
            boot_params.append(res.x)
        except:
            continue

    if len(boot_params) < 50:
        return [np.nan]*4

    boot_params = np.array(boot_params)
    return np.std(boot_params, axis=0)


# ── Chow test ─────────────────────────────────────────────────────
def chow_test(df, break_date, dep="delta_ff", regressors=["inflation","unemp_gap"]):
    sub = df[[dep] + regressors].dropna()
    before = sub.loc[:break_date]
    after = sub.loc[break_date:]
    if len(before) < 10 or len(after) < 10:
        return np.nan, np.nan
    k = len(regressors) + 1
    X_pool = sm.add_constant(sub[regressors].values); y_pool = sub[dep].values
    res_pool = sm.OLS(y_pool, X_pool).fit()
    X1 = sm.add_constant(before[regressors].values); y1 = before[dep].values
    res1 = sm.OLS(y1, X1).fit()
    X2 = sm.add_constant(after[regressors].values); y2 = after[dep].values
    res2 = sm.OLS(y2, X2).fit()
    n = len(sub)
    F = ((res_pool.ssr - res1.ssr - res2.ssr) / k) / ((res1.ssr + res2.ssr) / (n - 2*k))
    p_val = 1 - stats.f.cdf(F, k, n - 2*k)
    return F, p_val


# ── Expanding-window Taylor prescription ───────────────────────────
def expanding_taylor(df, min_window=60):
    dates_out, prescribed_out, actual_out = [], [], []
    for end in range(min_window, len(df)):
        window = df.iloc[:end+1]
        current = df.iloc[end]
        try:
            y = window["fed_funds"].values
            pi = window["inflation"].values
            u_gap = window["unemp_gap"].values
            ff_lag = window["ff_lag1"].values
            res = least_squares(taylor_resid, [0.85, 2.0, 0.5, 0.5],
                args=(y, pi, u_gap, ff_lag),
                bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]),
                max_nfev=500)
            rho, rstar, a_pi, a_u = res.x
            target = rstar + current["inflation"] + a_pi*(current["inflation"]-2.0) + a_u*current["unemp_gap"]
            prescribed_out.append(max(0, target))
        except:
            prescribed_out.append(np.nan)
        dates_out.append(df.index[end])
        actual_out.append(current["fed_funds"])
    return pd.DataFrame({"date": dates_out, "prescribed": prescribed_out,
                          "actual": actual_out}).set_index("date")


# ── Main analysis ──────────────────────────────────────────────────
def run_analysis(df, regime_df, regime_names, n_regimes):
    print("="*70)
    print("REGIME-CONDITIONAL TAYLOR RULES")
    print("  Using CBO NAIRU for unemployment gap")
    print("="*70)

    sub = regime_df[["fed_funds","ff_lag1","inflation","unemployment",
                     "unemp_gap","pi_expected","regime"]].dropna(
                         subset=["fed_funds","ff_lag1","inflation","unemp_gap"])

    # ── A. Full-sample Taylor rule (realized inflation) ──
    print("\n--- A. Full-sample structural Taylor rule (realized inflation, CBO NAIRU) ---")
    full = fit_taylor_nls(sub, "Full sample (realized pi)")
    print(f"\n  {'Parameter':<20s}  {'Estimate':>10s}  {'OLS SE':>8s}")
    print(f"  {'-'*42}")
    for nm, v, se in zip(["rho (smoothing)","r* (neutral)","a_pi (inflation)","a_u (unemp gap)"],
                          [full["rho"],full["rstar"],full["a_pi"],full["a_u"]], full["se"]):
        print(f"  {nm:<20s}  {v:>10.3f}  {se:>8.3f}")
    print(f"  Taylor principle: 1 + a_pi = {full['taylor_principle']:.2f}")

    # HAC standard errors
    print("\n  Computing block bootstrap HAC standard errors (500 reps)...")
    hac_se = bootstrap_taylor_se(sub, n_boot=500, block_size=12, use_expected=False)
    full["hac_se"] = hac_se
    print(f"  {'Parameter':<20s}  {'Estimate':>10s}  {'OLS SE':>8s}  {'HAC SE':>8s}")
    print(f"  {'-'*52}")
    for nm, v, se, hse in zip(["rho","r*","a_pi","a_u"],
                               [full["rho"],full["rstar"],full["a_pi"],full["a_u"]],
                               full["se"], hac_se):
        print(f"  {nm:<20s}  {v:>10.3f}  {se:>8.3f}  {hse:>8.3f}")

    # ── A2. Expected inflation specification ──
    print("\n--- A2. Full-sample Taylor rule (expected inflation, CBO NAIRU) ---")
    full_exp = fit_taylor_nls(sub, "Full sample (expected pi)", use_expected=True)
    if full_exp is not None:
        print(f"\n  {'Parameter':<20s}  {'Estimate':>10s}  {'SE':>8s}")
        print(f"  {'-'*42}")
        for nm, v, se in zip(["rho","r*","a_pi","a_u"],
                              [full_exp["rho"],full_exp["rstar"],full_exp["a_pi"],full_exp["a_u"]],
                              full_exp["se"]):
            print(f"  {nm:<20s}  {v:>10.3f}  {se:>8.3f}")
        print(f"  Taylor principle (expected pi): 1 + a_pi = {full_exp['taylor_principle']:.2f}")

        hac_se_exp = bootstrap_taylor_se(sub, n_boot=500, block_size=12, use_expected=True)
        full_exp["hac_se"] = hac_se_exp
        print(f"\n  HAC SE: {['%.3f' % s for s in hac_se_exp]}")
    else:
        print("  Not enough expected inflation data.")

    # ── B. Regime-conditional Taylor rules ──
    print(f"\n--- B. Regime-conditional Taylor rules ---")
    regime_results = {}
    regime_results_exp = {}
    for r in range(n_regimes):
        rsub = sub[sub["regime"] == r]
        if len(rsub) < 30:
            print(f"  {regime_names[r]}: too few obs ({len(rsub)}), skipping")
            continue

        # Realized inflation
        result = fit_taylor_nls(rsub, regime_names[r])
        regime_results[r] = result
        hac = bootstrap_taylor_se(rsub, n_boot=500, block_size=12)
        result["hac_se"] = hac

        # Expected inflation
        result_exp = fit_taylor_nls(rsub, f"{regime_names[r]} (exp pi)", use_expected=True)
        if result_exp is not None:
            regime_results_exp[r] = result_exp
            hac_exp = bootstrap_taylor_se(rsub, n_boot=500, block_size=12, use_expected=True)
            result_exp["hac_se"] = hac_exp

    if regime_results:
        print(f"\n  === Realized inflation specification ===")
        print(f"  {'Regime':<30s}  {'rho':>6s}  {'r*':>6s}  {'a_pi':>6s}  {'a_u':>6s}  {'1+a_pi':>7s}  {'N':>4s}")
        print(f"  {'-'*70}")
        for r, res in regime_results.items():
            tp = res["taylor_principle"]
            print(f"  {res['label']:<30s}  {res['rho']:>6.3f}  {res['rstar']:>6.3f}  "
                  f"{res['a_pi']:>6.3f}  {res['a_u']:>6.3f}  {tp:>7.2f}  {res['n']:>4d}")

        print(f"\n  HAC standard errors (block bootstrap, 12-month blocks):")
        print(f"  {'Regime':<30s}  {'se(rho)':>8s}  {'se(r*)':>8s}  {'se(a_pi)':>8s}  {'se(a_u)':>8s}")
        print(f"  {'-'*70}")
        for r, res in regime_results.items():
            hse = res["hac_se"]
            print(f"  {res['label']:<30s}  {hse[0]:>8.3f}  {hse[1]:>8.3f}  {hse[2]:>8.3f}  {hse[3]:>8.3f}")

    if regime_results_exp:
        print(f"\n  === Expected inflation specification ===")
        print(f"  {'Regime':<30s}  {'rho':>6s}  {'r*':>6s}  {'a_pi':>6s}  {'a_u':>6s}  {'1+a_pi':>7s}  {'N':>4s}")
        print(f"  {'-'*70}")
        for r, res in regime_results_exp.items():
            tp = res["taylor_principle"]
            print(f"  {res['label']:<30s}  {res['rho']:>6.3f}  {res['rstar']:>6.3f}  "
                  f"{res['a_pi']:>6.3f}  {res['a_u']:>6.3f}  {tp:>7.2f}  {res['n']:>4d}")

    # ── C. Structural break tests (using CBO NAIRU gap) ──
    print(f"\n--- C. Chow tests at regime transition dates ---")
    transitions = []
    prev_regime = regime_df["regime"].iloc[0]
    for i in range(1, len(regime_df)):
        curr = regime_df["regime"].iloc[i]
        if curr != prev_regime:
            transitions.append({
                "date": regime_df.index[i],
                "from": regime_names.get(prev_regime, f"R{prev_regime}"),
                "to": regime_names.get(curr, f"R{curr}"),
            })
        prev_regime = curr

    if transitions:
        tested_dates = set()
        print(f"\n  {'Date':<12s}  {'Transition':<40s}  {'F-stat':>8s}  {'p-value':>8s}")
        print(f"  {'-'*72}")
        for tr in transitions:
            d = tr["date"]
            if any(abs((d - td).days) < 365 for td in tested_dates):
                continue
            tested_dates.add(d)
            test_df = regime_df[["delta_ff","inflation","unemp_gap"]].dropna()
            F, p = chow_test(test_df, d.strftime("%Y-%m-%d"),
                             regressors=["inflation","unemp_gap"])
            sig = "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""
            print(f"  {d:%Y-%m}      {tr['from'][:18]:>18s} -> {tr['to'][:18]:<18s}  {F:>8.2f}  {p:>7.4f} {sig}")

    # ── D. Expanding-window Taylor prescription ──
    print(f"\n--- D. Expanding-window Taylor rule prescription ---")
    expand_df = expanding_taylor(regime_df, min_window=60)
    expand_df["gap"] = expand_df["actual"] - expand_df["prescribed"]
    expand_df["regime"] = regime_df.loc[expand_df.index, "regime"]

    print(f"  Computed {len(expand_df)} real-time prescriptions")
    print(f"  Mean discretionary gap: {expand_df['gap'].mean():+.2f} pp")
    for r in range(n_regimes):
        rsub = expand_df[expand_df["regime"] == r]
        if len(rsub) > 0:
            print(f"  {regime_names[r]}: mean gap = {rsub['gap'].mean():+.2f}")

    return full, full_exp, regime_results, regime_results_exp, expand_df, transitions


# ── Visualization ──────────────────────────────────────────────────
def plot_results(regime_df, regime_names, n_regimes, full, full_exp,
                 regime_results, regime_results_exp, expand_df, output_path):
    regime_colors = [C5, C1, C4, C2, "#94a3b8"][:n_regimes]

    # ── Fig 1: Regime-conditional Taylor parameters (with HAC SE) ──
    if len(regime_results) >= 2:
        fig, axes = plt.subplots(1, 4, figsize=(16, 5))
        fig.suptitle("Taylor Rule Parameters by Inflation Regime (CBO NAIRU, HAC SE)",
                     fontweight="bold", fontsize=14, y=1.02)

        params = ["rho", "rstar", "a_pi", "a_u"]
        titles = ["Smoothing (rho)", "Neutral rate (r*)",
                  "Inflation response (a_pi)", "Unemp gap response (a_u)"]

        for ax, param, title in zip(axes, params, titles):
            labels_list = []; vals = []; colors = []; errors = []
            idx = params.index(param)

            # Full sample
            labels_list.append("Full sample"); vals.append(full[param])
            colors.append("#6b7280")
            hse = full.get("hac_se", full["se"])
            errors.append(1.96 * hse[idx] if not np.isnan(hse[idx]) else 0)

            for r in sorted(regime_results.keys()):
                res = regime_results[r]
                labels_list.append(regime_names[r][:20]); vals.append(res[param])
                colors.append(regime_colors[r])
                hse_r = res.get("hac_se", res["se"])
                errors.append(1.96 * hse_r[idx] if not np.isnan(hse_r[idx]) else 0)

            y_pos = np.arange(len(labels_list))
            ax.barh(y_pos, vals, xerr=errors, color=colors, alpha=0.75,
                    edgecolor="white", lw=1, capsize=4, height=0.6)
            ax.set_yticks(y_pos); ax.set_yticklabels(labels_list, fontsize=8)
            ax.axvline(0, color="grey", lw=0.5)
            ax.set_title(title, fontweight="bold", fontsize=10)
            ax.invert_yaxis()
            if param == "a_pi":
                ax.axvline(0, color=C2, ls="--", lw=1, alpha=0.5)

        fig.tight_layout()
        path1 = f"{output_path}/taylor_by_regime.png"
        fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path1}")
    else:
        path1 = None

    # ── Fig 2: Taylor prescription vs actual ──
    fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True,
                              gridspec_kw={"height_ratios": [1, 3, 2]})
    fig.suptitle("Taylor Rule Prescription vs Actual Fed Funds Rate (CBO NAIRU)",
                 fontweight="bold", fontsize=14, y=0.995)

    ax = axes[0]
    for r in range(n_regimes):
        mask = regime_df.loc[expand_df.index, "regime"] == r
        for d in expand_df.index[mask]:
            ax.axvspan(d, d + pd.DateOffset(months=1), color=regime_colors[r], alpha=0.8)
    ax.set_yticks([])
    ax.set_title("Inflation regime", fontweight="bold", fontsize=10)
    legend_elements = [Patch(facecolor=regime_colors[r], alpha=0.8, label=regime_names[r])
                       for r in range(n_regimes)]
    ax.legend(handles=legend_elements, frameon=True, fontsize=7, loc="upper left",
              ncol=min(n_regimes,3))

    ax = axes[1]; shade_recessions(ax)
    ax.plot(expand_df.index, expand_df["actual"], lw=2, color=C1, label="Actual fed funds", zorder=3)
    ax.plot(expand_df.index, expand_df["prescribed"], lw=1.5, color=C2, alpha=0.8,
            label="Taylor rule prescription", zorder=2)
    ax.fill_between(expand_df.index, expand_df["actual"], expand_df["prescribed"],
                    where=expand_df["actual"] > expand_df["prescribed"],
                    alpha=0.15, color=C2, label="Too tight")
    ax.fill_between(expand_df.index, expand_df["actual"], expand_df["prescribed"],
                    where=expand_df["actual"] < expand_df["prescribed"],
                    alpha=0.15, color=C1, label="Too loose")
    ax.set_ylabel("Rate (%)")
    ax.set_title("Fed funds: actual vs Taylor rule prescription", fontweight="bold")
    ax.legend(frameon=True, fontsize=8, loc="upper right")
    ax.set_ylim(bottom=-1)

    ax = axes[2]; shade_recessions(ax)
    gap = expand_df["gap"]
    ax.fill_between(expand_df.index, gap, 0, where=gap > 0, alpha=0.4, color=C2, label="Tighter than rule")
    ax.fill_between(expand_df.index, gap, 0, where=gap < 0, alpha=0.4, color=C1, label="Looser than rule")
    ax.axhline(0, color="grey", lw=1)
    ax.set_ylabel("Gap (pp)"); ax.set_xlabel("Year")
    ax.set_title("Discretionary gap (actual minus prescribed)", fontweight="bold")
    ax.legend(frameon=True, fontsize=8)

    fig.tight_layout()
    path2 = f"{output_path}/taylor_prescription_vs_actual.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # ── Fig 3: Taylor principle by regime (realized vs expected) ──
    if len(regime_results) >= 2:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle("Taylor Principle: Realized vs Expected Inflation",
                     fontweight="bold", fontsize=14, y=1.02)

        for ax, (results_dict, title_suffix) in zip(axes, [
            (regime_results, "Realized inflation"),
            (regime_results_exp, "Expected inflation (MICH/adaptive)")]):

            if not results_dict:
                ax.text(0.5, 0.5, "Not enough data", ha="center", va="center",
                        transform=ax.transAxes)
                ax.set_title(title_suffix, fontweight="bold")
                continue

            labels_list = ["Full sample"] + [regime_names[r][:25] for r in sorted(results_dict.keys())]
            ref = full if "Realized" in title_suffix else full_exp
            if ref is None:
                continue
            tp_vals = [ref["taylor_principle"]] + \
                      [results_dict[r]["taylor_principle"] for r in sorted(results_dict.keys())]
            colors_bar = ["#6b7280"] + [regime_colors[r] for r in sorted(results_dict.keys())]

            # HAC-based error bars
            errs = []
            hse = ref.get("hac_se", ref["se"])
            errs.append(1.96 * hse[2] if not np.isnan(hse[2]) else 0)
            for r in sorted(results_dict.keys()):
                hse_r = results_dict[r].get("hac_se", results_dict[r]["se"])
                errs.append(1.96 * hse_r[2] if not np.isnan(hse_r[2]) else 0)

            bars = ax.bar(labels_list, tp_vals, yerr=errs, color=colors_bar,
                          alpha=0.75, edgecolor="white", lw=1, width=0.6, capsize=5)
            ax.axhline(1.0, color=C2, ls="--", lw=2, alpha=0.7, label="Taylor principle = 1")
            ax.set_ylabel("1 + a_pi")
            ax.set_title(title_suffix, fontweight="bold")
            ax.legend(frameon=False, fontsize=9)
            for bar, val in zip(bars, tp_vals):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(errs)*0.1 + 0.02,
                        f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

        fig.tight_layout()
        path3 = f"{output_path}/taylor_principle_by_regime.png"
        fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path3}")
    else:
        path3 = None

    return [p for p in [path1, path2, path3] if p]


# ── Main ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    print("Fitting Markov-switching regimes...")
    regime_df, regime_names, n_regimes = get_regimes(df, n_regimes=2)
    print(f"  {n_regimes} regimes: {regime_names}")

    full, full_exp, regime_results, regime_results_exp, expand_df, transitions = \
        run_analysis(df, regime_df, regime_names, n_regimes)

    paths = plot_results(regime_df, regime_names, n_regimes, full, full_exp,
                         regime_results, regime_results_exp, expand_df, OUTPUT_PATH)

    print(f"\nOutputs: {paths}")
    print("Done.")

Loading data...
Merged: 857 obs (1954-07 to 2025-12)
Fitting Markov-switching regimes...
  2 regimes: {0: 'Low inflation (2.4%)', 1: 'High inflation (7.4%)'}
REGIME-CONDITIONAL TAYLOR RULES
  Using CBO NAIRU for unemployment gap

--- A. Full-sample structural Taylor rule (realized inflation, CBO NAIRU) ---

  Parameter               Estimate    OLS SE
  ------------------------------------------
  rho (smoothing)            0.974     0.007
  r* (neutral)               1.901     0.805
  a_pi (inflation)          -0.075     0.237
  a_u (unemp gap)           -1.594     0.534
  Taylor principle: 1 + a_pi = 0.92

  Computing block bootstrap HAC standard errors (500 reps)...
  Parameter               Estimate    OLS SE    HAC SE
  ----------------------------------------------------
  rho                        0.974     0.007     0.010
  r*                         1.901     0.805     1.126
  a_pi                      -0.075     0.237     0.669
  a_u                       -1.594     0.534   

In [13]:
#!/usr/bin/env python3
"""
Component 3: Structural Break Detection & Yield Curve Analysis
==============================================================
Updated: uses CBO NAIRU-based unemployment gap in break detection.
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from scipy import stats

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]
FED_CHAIRS = [
    ("Burns","1970-02","1978-01"),("Miller","1978-03","1979-08"),
    ("Volcker","1979-08","1987-08"),("Greenspan","1987-08","2006-01"),
    ("Bernanke","2006-02","2014-01"),("Yellen","2014-02","2018-02"),
    ("Powell","2018-02","2024-12"),
]
CHAIR_DATES = [(ch, pd.Timestamp(s), pd.Timestamp(e)) for ch,s,e in FED_CHAIRS]

def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else:
        merged["expected_inflation"] = np.nan
    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["delta_pi"] = df["inflation"].diff()
    df["delta_u"] = df["unemployment"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation","unemp_gap","capacity_gap","fed_funds","term_spread","fin_conditions"]:
        for lag in [1,3,6,12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    for h in [1,3,6,12]:
        df[f"inflation_{h}m_ahead"] = df["inflation"].shift(-h)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation","L12_inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


def get_regimes(df, n_regimes=2):
    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())
    mod = sm.tsa.MarkovRegression(endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True)
    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf: best_llf = res.llf; best_res = res
        except: continue
    smoothed = best_res.smoothed_marginal_probabilities
    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0
    sorted_regimes = sorted(regime_means, key=regime_means.get)
    rdf = df.copy()
    for new_r in range(n_regimes):
        rdf[f"p_regime_{new_r}"] = smoothed[sorted_regimes[new_r]].values
    rdf["regime"] = np.argmax(
        np.column_stack([rdf[f"p_regime_{r}"].values for r in range(n_regimes)]), axis=1)
    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    labels = ["Low","High"] if n_regimes==2 else [f"R{r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}
    return rdf, regime_names, n_regimes


# ================================================================
# PART A: STRUCTURAL BREAK DETECTION (now uses CBO NAIRU gap)
# ================================================================
def sup_wald_break_test(df, dep="delta_ff", regressors=["inflation","unemp_gap"],
                        min_frac=0.15):
    sub = df[[dep] + regressors].dropna()
    n = len(sub); trim = int(n * min_frac); k = len(regressors) + 1
    X_pool = sm.add_constant(sub[regressors].values)
    y_pool = sub[dep].values
    res_pool = sm.OLS(y_pool, X_pool).fit()
    ssr_pool = res_pool.ssr

    dates_test, f_stats = [], []
    for t in range(trim, n - trim):
        X1 = sm.add_constant(sub[regressors].values[:t])
        y1 = sub[dep].values[:t]
        X2 = sm.add_constant(sub[regressors].values[t:])
        y2 = sub[dep].values[t:]
        try:
            res1 = sm.OLS(y1, X1).fit(); res2 = sm.OLS(y2, X2).fit()
            F = ((ssr_pool - res1.ssr - res2.ssr) / k) / ((res1.ssr + res2.ssr) / (n - 2*k))
            f_stats.append(F)
        except:
            f_stats.append(0)
        dates_test.append(sub.index[t])

    f_stats = np.array(f_stats)
    dates_test = pd.DatetimeIndex(dates_test)
    best_idx = np.argmax(f_stats)

    # Andrews (1993) critical values for sup-Wald, k=3, 15% trimming
    cv_10 = 7.12; cv_05 = 8.68; cv_01 = 12.16
    best_F = f_stats[best_idx]
    sig = "***" if best_F > cv_01 else "**" if best_F > cv_05 else "*" if best_F > cv_10 else ""

    return {
        "best_date": dates_test[best_idx], "best_F": best_F, "sig": sig,
        "cv": {"1%": cv_01, "5%": cv_05, "10%": cv_10},
        "dates": dates_test, "f_stats": f_stats,
    }


def sequential_breaks(df, dep="delta_ff", regressors=["inflation","unemp_gap"],
                      max_breaks=4, min_frac=0.15):
    print(f"\n{'='*70}")
    print("STRUCTURAL BREAK DETECTION (Sequential sup-Wald)")
    print(f"  Using CBO NAIRU-based unemployment gap")
    print(f"{'='*70}")
    print(f"  DV: {dep}  |  Regressors: {regressors}")

    sub = df[[dep] + regressors].dropna()
    breaks = []
    segments = [(sub.index[0], sub.index[-1])]

    for b in range(max_breaks):
        best_overall = None
        for seg_start, seg_end in segments:
            seg = sub.loc[seg_start:seg_end]
            if len(seg) < int(len(sub) * min_frac * 2):
                continue
            result = sup_wald_break_test(seg, dep, regressors, min_frac)
            if result["best_F"] > result["cv"]["5%"]:
                if best_overall is None or result["best_F"] > best_overall["best_F"]:
                    best_overall = result
                    best_overall["segment"] = (seg_start, seg_end)

        if best_overall is None:
            print(f"\n  Break {b+1}: no further significant breaks found.")
            break

        breaks.append(best_overall)
        bd = best_overall["best_date"]
        print(f"\n  Break {b+1}: {bd:%Y-%m}  (F = {best_overall['best_F']:.2f}{best_overall['sig']})")

        seg_s, seg_e = best_overall["segment"]
        new_segments = []
        for s, e in segments:
            if s == seg_s and e == seg_e:
                new_segments.append((s, bd)); new_segments.append((bd, e))
            else:
                new_segments.append((s, e))
        segments = new_segments

    print(f"\n  Detected breaks vs chair transitions:")
    print(f"  {'Break date':<14s}  {'Nearest chair transition':<30s}  {'Distance':>10s}")
    print(f"  {'-'*58}")
    for brk in breaks:
        bd = brk["best_date"]
        nearest = min(CHAIR_DATES, key=lambda x: min(abs((bd-x[1]).days), abs((bd-x[2]).days)))
        d1 = abs((bd - nearest[1]).days); d2 = abs((bd - nearest[2]).days)
        if d1 < d2:
            chair_event = f"{nearest[0]} start ({nearest[1]:%Y-%m})"; dist_months = d1/30
        else:
            chair_event = f"{nearest[0]} end ({nearest[2]:%Y-%m})"; dist_months = d2/30
        print(f"  {bd:%Y-%m}        {chair_event:<30s}  {dist_months:>8.1f}m")

    return breaks, segments


# ================================================================
# PART B: YIELD CURVE AND REGIME TRANSITIONS
# ================================================================
def yield_curve_regime_analysis(regime_df, regime_names, n_regimes):
    print(f"\n{'='*70}")
    print("YIELD CURVE AND REGIME TRANSITIONS")
    print(f"{'='*70}")

    transitions = []
    prev = regime_df["regime"].iloc[0]
    for i in range(1, len(regime_df)):
        curr = regime_df["regime"].iloc[i]
        if curr != prev:
            transitions.append({"date": regime_df.index[i], "from_regime": prev,
                                "to_regime": curr,
                                "direction": "up" if curr > prev else "down"})
        prev = curr

    filtered = []
    for tr in transitions:
        if not filtered or (tr["date"] - filtered[-1]["date"]).days > 365:
            filtered.append(tr)
    transitions = filtered

    print(f"\n  Major regime transitions: {len(transitions)}")
    print(f"\n  {'Date':<12s}  {'Direction':<8s}  {'Spread at t':>12s}  {'Spread t-6':>12s}  {'Spread t-12':>12s}")
    print(f"  {'-'*60}")

    for tr in transitions:
        d = tr["date"]
        spread_at = regime_df.loc[:d, "term_spread"].iloc[-1]
        d_6 = d - pd.DateOffset(months=6)
        mask_6 = (regime_df.index >= d_6) & (regime_df.index < d)
        spread_6 = regime_df.loc[mask_6, "term_spread"].mean() if mask_6.sum() > 0 else np.nan
        d_12 = d - pd.DateOffset(months=12)
        mask_12 = (regime_df.index >= d_12) & (regime_df.index < d)
        spread_12 = regime_df.loc[mask_12, "term_spread"].mean() if mask_12.sum() > 0 else np.nan
        tr["spread_at"] = spread_at; tr["spread_6m"] = spread_6; tr["spread_12m"] = spread_12
        print(f"  {d:%Y-%m}      {tr['direction']:<8s}  {spread_at:>12.2f}  {spread_6:>12.2f}  {spread_12:>12.2f}")

    # Logit test
    rdf = regime_df.copy()
    rdf["regime_change"] = (rdf["regime"].diff() != 0).astype(int)
    rdf["regime_change_12m"] = rdf["regime_change"].rolling(12).max().shift(-12)
    test_df = rdf[["term_spread","regime_change_12m"]].dropna()
    if len(test_df) > 50:
        X = sm.add_constant(test_df["term_spread"].values)
        y = test_df["regime_change_12m"].values
        try:
            logit = sm.Logit(y, X).fit(disp=False)
            print(f"\n  Logit: P(regime change in 12m) ~ term_spread")
            print(f"  Spread coefficient: {logit.params[1]:.4f} (p={logit.pvalues[1]:.4f})")
            if logit.pvalues[1] < 0.05:
                print(f"  -> Term spread significantly predicts regime transitions")
            else:
                print(f"  -> Term spread does NOT significantly predict regime transitions")
        except:
            pass

    return transitions


# ================================================================
# PART C: RECESSION MODEL ROBUSTNESS
# ================================================================
def leave_one_recession_out(df):
    print(f"\n{'='*70}")
    print("RECESSION MODEL: LEAVE-ONE-RECESSION-OUT CV")
    print(f"{'='*70}")

    rec_features = ["term_spread","inflation","unemp_gap","fed_funds",
                    "capacity_gap","fin_conditions","real_rate"]

    df = df.copy()
    for h in [6,12,18]:
        col = f"recession_{h}m_ahead"
        if col not in df.columns:
            df[col] = df["recession"].rolling(h).max().shift(-h)

    recession_episodes = [
        ("1973-74 Oil", "1973-11", "1975-03"),
        ("1980 Volcker I", "1980-01", "1980-07"),
        ("1981-82 Volcker II", "1981-07", "1982-11"),
        ("1990-91", "1990-07", "1991-03"),
        ("2001 Dot-com", "2001-03", "2001-11"),
        ("2007-09 GFC", "2007-12", "2009-06"),
        ("2020 COVID", "2020-02", "2020-04"),
    ]

    for h in [6, 12]:
        target = f"recession_{h}m_ahead"
        rec_df = df[rec_features + [target]].dropna()
        y_full = rec_df[target].astype(int).values
        X_full = rec_df[rec_features].values

        print(f"\n  --- {h}-month horizon ---")
        print(f"  {'Left out':<22s}  {'Train N':>8s}  {'Test N':>8s}  {'AUC (full)':>10s}  {'AUC (spread)':>12s}")
        print(f"  {'-'*66}")

        tr_mask = rec_df.index <= "2015-12"; te_mask = ~tr_mask
        if te_mask.sum() > 0 and len(np.unique(y_full[te_mask])) > 1:
            sc = StandardScaler(); X_tr = sc.fit_transform(X_full[tr_mask]); X_te = sc.transform(X_full[te_mask])
            clf = LogisticRegression(C=1.0, max_iter=1000, random_state=SEED)
            clf.fit(X_tr, y_full[tr_mask])
            ref_auc = roc_auc_score(y_full[te_mask], clf.predict_proba(X_te)[:,1])
            print(f"  {'Time-split ref':<22s}  {tr_mask.sum():>8d}  {te_mask.sum():>8d}  {ref_auc:>10.3f}")
        print()

        results_loro = []
        for name, rs, re in recession_episodes:
            pre_start = pd.Timestamp(rs) - pd.DateOffset(months=18)
            test_mask = (rec_df.index >= pre_start) & (rec_df.index <= re)
            train_mask = ~test_mask
            y_tr = y_full[train_mask]; X_tr_raw = X_full[train_mask]
            y_te = y_full[test_mask]; X_te_raw = X_full[test_mask]
            if len(y_te) < 5 or len(np.unique(y_te)) < 2:
                print(f"  {name:<22s}  {train_mask.sum():>8d}  {test_mask.sum():>8d}  {'skip':>10s}")
                continue
            sc = StandardScaler()
            X_tr = sc.fit_transform(X_tr_raw); X_te = sc.transform(X_te_raw)
            clf_full = LogisticRegression(C=1.0, max_iter=1000, random_state=SEED)
            clf_full.fit(X_tr, y_tr)
            auc_full = roc_auc_score(y_te, clf_full.predict_proba(X_te)[:,1])
            si = rec_features.index("term_spread")
            clf_sp = LogisticRegression(C=1.0, max_iter=1000, random_state=SEED)
            clf_sp.fit(X_tr[:,si:si+1], y_tr)
            auc_sp = roc_auc_score(y_te, clf_sp.predict_proba(X_te[:,si:si+1])[:,1])
            results_loro.append({"name": name, "auc_full": auc_full, "auc_spread": auc_sp})
            print(f"  {name:<22s}  {train_mask.sum():>8d}  {test_mask.sum():>8d}  {auc_full:>10.3f}  {auc_sp:>12.3f}")

        if results_loro:
            avg_full = np.mean([r["auc_full"] for r in results_loro])
            avg_sp = np.mean([r["auc_spread"] for r in results_loro])
            print(f"\n  {'Average LORO':<22s}  {'':>8s}  {'':>8s}  {avg_full:>10.3f}  {avg_sp:>12.3f}")

    return results_loro


# ================================================================
# VISUALIZATION
# ================================================================
def plot_results(df, regime_df, regime_names, n_regimes, breaks, transitions, output_path):
    regime_colors = [C5, C1, C4, C2][:n_regimes]
    chair_colors_map = {"Burns":"#dbeafe","Miller":"#fee2e2","Volcker":"#ede9fe",
                        "Greenspan":"#fef3c7","Bernanke":"#d1fae5","Yellen":"#fce7f3","Powell":"#e0e7ff"}

    # ── Fig 1: Sup-Wald F-statistic ──
    if breaks:
        first_break = breaks[0]
        fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True,
                                  gridspec_kw={"height_ratios": [1, 2, 2]})
        fig.suptitle("Structural Break Detection in Fed Reaction Function (CBO NAIRU gap)",
                     fontweight="bold", fontsize=14, y=0.995)

        ax = axes[0]
        for ch, cs, ce in CHAIR_DATES:
            color = chair_colors_map.get(ch, "#e5e7eb")
            ax.axvspan(cs, ce, color=color, alpha=0.5)
            mid = cs + (ce - cs)/2
            if df.index[0] <= mid <= df.index[-1]:
                ax.text(mid, 0.5, ch, ha="center", va="center", fontsize=8, fontstyle="italic")
        ax.set_yticks([]); ax.set_title("Fed chair tenures", fontweight="bold", fontsize=10)

        ax = axes[1]
        ax.plot(first_break["dates"], first_break["f_stats"], lw=2, color=C1)
        ax.axhline(first_break["cv"]["5%"], color=C2, ls="--", lw=1.5,
                    label=f'5% critical value ({first_break["cv"]["5%"]:.1f})')
        ax.axhline(first_break["cv"]["1%"], color=C2, ls=":", lw=1,
                    label=f'1% critical value ({first_break["cv"]["1%"]:.1f})')
        for brk in breaks:
            ax.axvline(brk["best_date"], color=C3, ls="-", lw=2, alpha=0.7)
            ax.text(brk["best_date"], ax.get_ylim()[1]*0.9,
                    f' {brk["best_date"]:%Y-%m}', fontsize=9, color=C3, fontweight="bold")
        for ch, cs, ce in CHAIR_DATES:
            if first_break["dates"][0] <= cs <= first_break["dates"][-1]:
                ax.axvline(cs, color="#9ca3af", ls="--", lw=1, alpha=0.5)
        ax.set_ylabel("F-statistic")
        ax.set_title("Sup-Wald F-statistic (testing every possible break date)", fontweight="bold")
        ax.legend(frameon=True, fontsize=8)

        ax = axes[2]; shade_recessions(ax)
        test_df = df[["delta_ff"]].dropna()
        ax.plot(test_df.index, test_df["delta_ff"], lw=1, color="#374151", alpha=0.7)
        for brk in breaks:
            ax.axvline(brk["best_date"], color=C3, ls="-", lw=2, alpha=0.7)
        ax.axhline(0, color="grey", lw=0.8)
        ax.set_ylabel("Monthly change (pp)")
        ax.set_title("Fed funds rate changes (dependent variable)", fontweight="bold")
        ax.set_xlabel("Year")

        fig.tight_layout()
        path1 = f"{output_path}/structural_breaks.png"
        fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path1}")
    else:
        path1 = None

    # ── Fig 2: Term spread around regime transitions ──
    if transitions:
        fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)
        fig.suptitle("Yield Curve Behaviour Around Inflation Regime Transitions",
                     fontweight="bold", fontsize=14, y=0.995)

        ax = axes[0]; shade_recessions(ax)
        ax.plot(regime_df.index, regime_df["term_spread"], lw=1.5, color=C1)
        ax.axhline(0, color=C2, ls="--", lw=1.5, alpha=0.7, label="Inversion threshold")
        for tr in transitions:
            color = C2 if tr["direction"] == "up" else C5
            ax.axvline(tr["date"], color=color, ls="-", lw=2, alpha=0.7)
        ax.set_ylabel("Term spread (pp)")
        ax.set_title("10Y-FFR spread with regime transition dates", fontweight="bold")
        legend_elements = [
            Line2D([0],[0], color=C2, ls="-", lw=2, label="Transition to high inflation"),
            Line2D([0],[0], color=C5, ls="-", lw=2, label="Transition to low inflation"),
            Line2D([0],[0], color=C2, ls="--", lw=1.5, label="Inversion threshold"),
        ]
        ax.legend(handles=legend_elements, frameon=True, fontsize=8)

        ax = axes[1]
        for r in range(n_regimes):
            mask = regime_df["regime"] == r
            for d in regime_df.index[mask]:
                ax.axvspan(d, d + pd.DateOffset(months=1), color=regime_colors[r], alpha=0.8)
        ax.set_yticks([]); ax.set_xlabel("Year")
        ax.set_title("Inflation regime", fontweight="bold")
        legend_elements = [Patch(facecolor=regime_colors[r], alpha=0.8, label=regime_names[r])
                           for r in range(n_regimes)]
        ax.legend(handles=legend_elements, frameon=True, fontsize=7, loc="upper left")

        fig.tight_layout()
        path2 = f"{output_path}/yield_curve_regimes.png"
        fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path2}")
    else:
        path2 = None

    return [p for p in [path1, path2] if p]


# ── Main ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    print("Fitting regimes...")
    regime_df, regime_names, n_regimes = get_regimes(df, n_regimes=2)
    print(f"  {n_regimes} regimes: {regime_names}")

    breaks, segments = sequential_breaks(
        regime_df, dep="delta_ff", regressors=["inflation","unemp_gap"],
        max_breaks=3, min_frac=0.15)

    transitions = yield_curve_regime_analysis(regime_df, regime_names, n_regimes)
    loro_results = leave_one_recession_out(df)

    paths = plot_results(df, regime_df, regime_names, n_regimes,
                         breaks, transitions, OUTPUT_PATH)

    print(f"\nOutputs: {paths}")
    print("Done.")

Loading data...
Merged: 857 obs (1954-07 to 2025-12)
Fitting regimes...
  2 regimes: {0: 'Low inflation (2.4%)', 1: 'High inflation (7.4%)'}

STRUCTURAL BREAK DETECTION (Sequential sup-Wald)
  Using CBO NAIRU-based unemployment gap
  DV: delta_ff  |  Regressors: ['inflation', 'unemp_gap']

  Break 1: no further significant breaks found.

  Detected breaks vs chair transitions:
  Break date      Nearest chair transition          Distance
  ----------------------------------------------------------

YIELD CURVE AND REGIME TRANSITIONS

  Major regime transitions: 9

  Date          Direction   Spread at t    Spread t-6   Spread t-12
  ------------------------------------------------------------
  1968-07      up               -0.53          0.29          0.85
  1971-06      down              1.61          1.90          1.35
  1973-03      up               -0.38          0.99          1.39
  1982-11      down              1.35          1.14          0.62
  1988-11      up                0.

In [14]:
#!/usr/bin/env python3
"""
Component 4: Regime-Aware Inflation Forecasting
================================================
Updated: uses CBO NAIRU-based unemployment gap throughout.
Removed synthetic data fallback.
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]

def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else:
        merged["expected_inflation"] = np.nan
    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation","unemp_gap","capacity_gap","fed_funds","term_spread","fin_conditions"]:
        for lag in [1,3,6,12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    for h in [1,3,6,12]:
        df[f"inflation_{h}m_ahead"] = df["inflation"].shift(-h)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation","L12_inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


def get_regimes(df, n_regimes=2):
    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())
    mod = sm.tsa.MarkovRegression(endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True)
    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf: best_llf = res.llf; best_res = res
        except: continue
    smoothed = best_res.smoothed_marginal_probabilities
    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0
    sorted_regimes = sorted(regime_means, key=regime_means.get)
    rdf = df.copy()
    for new_r in range(n_regimes):
        rdf[f"p_regime_{new_r}"] = smoothed[sorted_regimes[new_r]].values
    rdf["regime"] = np.argmax(
        np.column_stack([rdf[f"p_regime_{r}"].values for r in range(n_regimes)]), axis=1)
    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    labels = ["Low","High"] if n_regimes==2 else [f"R{r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}
    return rdf, regime_names, n_regimes


# ================================================================
# FORECASTING
# ================================================================

BASE_FEATURES = ["L1_inflation","L3_inflation","L6_inflation","L12_inflation"]
EXTENDED_FEATURES = BASE_FEATURES + [
    "L1_fed_funds","L3_fed_funds","L6_fed_funds",
    "L6_unemp_gap","L6_term_spread","L6_fin_conditions"
]

CV_FOLDS = [
    {"train_end":"1990-12","test_start":"1991-01","test_end":"1995-12","name":"Early 1990s"},
    {"train_end":"1995-12","test_start":"1996-01","test_end":"2000-12","name":"Late 1990s"},
    {"train_end":"2000-12","test_start":"2001-01","test_end":"2007-12","name":"2000s"},
    {"train_end":"2007-12","test_start":"2008-01","test_end":"2015-12","name":"GFC & after"},
    {"train_end":"2015-12","test_start":"2016-01","test_end":"2023-01","name":"Recent"},
]

HORIZONS = [1, 3, 6, 12]


def run_forecasting(regime_df, regime_names, n_regimes):
    print(f"\n{'='*70}")
    print("REGIME-AWARE INFLATION FORECASTING (CBO NAIRU gap)")
    print(f"{'='*70}")

    base_f = [f for f in BASE_FEATURES if f in regime_df.columns]
    ext_f = [f for f in EXTENDED_FEATURES if f in regime_df.columns]

    all_results = {}

    for h in HORIZONS:
        target = f"inflation_{h}m_ahead"
        if target not in regime_df.columns:
            regime_df[target] = regime_df["inflation"].shift(-h)

        fc_df = regime_df[ext_f + [target, "regime"] +
                          [f"p_regime_{r}" for r in range(n_regimes)]].dropna()

        print(f"\n  === {h}-month horizon ({len(fc_df)} obs) ===")

        models = {
            "AR (lags only)": {"features": base_f, "regime_aware": False,
                "fn": lambda: Ridge(alpha=1.0)},
            "Pooled Ridge": {"features": ext_f, "regime_aware": False,
                "fn": lambda: Ridge(alpha=10.0)},
            "Ridge + regime": {"features": ext_f + ["regime"], "regime_aware": False,
                "fn": lambda: Ridge(alpha=10.0)},
            "Ridge + regime prob": {"features": ext_f + [f"p_regime_{r}" for r in range(n_regimes)],
                "regime_aware": False, "fn": lambda: Ridge(alpha=10.0)},
            "Regime-conditional": {"features": ext_f, "regime_aware": True,
                "fn": lambda: Ridge(alpha=10.0)},
            "RF pooled": {"features": ext_f, "regime_aware": False,
                "fn": lambda: RandomForestRegressor(n_estimators=200, max_depth=6,
                    min_samples_leaf=10, random_state=SEED)},
        }

        horizon_results = {}

        for mname, spec in models.items():
            fold_results = []

            for fold in CV_FOLDS:
                tr = fc_df.loc[:fold["train_end"]]
                te = fc_df.loc[fold["test_start"]:fold["test_end"]]
                if len(te) < 6 or len(tr) < 30:
                    continue

                feats = [f for f in spec["features"] if f in fc_df.columns]

                if spec["regime_aware"]:
                    preds = np.full(len(te), np.nan)
                    for r in range(n_regimes):
                        tr_r = tr[tr["regime"] == r]
                        te_r_mask = te["regime"] == r
                        if te_r_mask.sum() == 0:
                            continue
                        if len(tr_r) < 15:
                            sc = StandardScaler()
                            Xtr = sc.fit_transform(tr[feats])
                            Xte = sc.transform(te.loc[te_r_mask, feats])
                            m = spec["fn"](); m.fit(Xtr, tr[target].values)
                            preds[te_r_mask.values] = m.predict(Xte)
                            continue
                        sc = StandardScaler()
                        Xtr = sc.fit_transform(tr_r[feats])
                        Xte = sc.transform(te.loc[te_r_mask, feats])
                        m = spec["fn"](); m.fit(Xtr, tr_r[target].values)
                        preds[te_r_mask.values] = m.predict(Xte)

                    valid = ~np.isnan(preds)
                    if valid.sum() < 6: continue
                    mse = mean_squared_error(te[target].values[valid], preds[valid])
                    r2 = r2_score(te[target].values[valid], preds[valid])
                    mae = mean_absolute_error(te[target].values[valid], preds[valid])
                else:
                    sc = StandardScaler()
                    Xtr = sc.fit_transform(tr[feats])
                    Xte = sc.transform(te[feats])
                    m = spec["fn"](); m.fit(Xtr, tr[target].values)
                    preds = m.predict(Xte)
                    mse = mean_squared_error(te[target].values, preds)
                    r2 = r2_score(te[target].values, preds)
                    mae = mean_absolute_error(te[target].values, preds)

                fold_results.append({"mse": mse, "r2": r2, "mae": mae, "fold": fold["name"]})

            if fold_results:
                horizon_results[mname] = {
                    "avg_mse": np.mean([f["mse"] for f in fold_results]),
                    "avg_r2": np.mean([f["r2"] for f in fold_results]),
                    "avg_mae": np.mean([f["mae"] for f in fold_results]),
                    "folds": fold_results
                }

        print(f"\n  {'Model':<25s}  {'MSE':>8s}  {'MAE':>8s}  {'R2':>8s}")
        print(f"  {'-'*53}")
        for mname in models:
            if mname in horizon_results:
                r = horizon_results[mname]
                print(f"  {mname:<25s}  {r['avg_mse']:>8.3f}  {r['avg_mae']:>8.3f}  {r['avg_r2']:>+8.3f}")

        all_results[h] = horizon_results

    return all_results


# ================================================================
# SUBSAMPLE STABILITY
# ================================================================
def subsample_stability(regime_df):
    print(f"\n{'='*70}")
    print("SUBSAMPLE STABILITY: WHY 12-MONTH FORECASTING FAILS")
    print(f"{'='*70}")

    print(f"\n  Inflation autocorrelation by regime:")
    print(f"  {'Regime':<25s}", end="")
    for lag in [1, 3, 6, 12]:
        print(f"  {'AC('+str(lag)+')':>7s}", end="")
    print()
    print(f"  {'-'*60}")

    for r in sorted(regime_df["regime"].unique()):
        sub = regime_df[regime_df["regime"] == r]["inflation"]
        print(f"  {'R'+str(r)+' (n='+str(len(sub))+')' :<25s}", end="")
        for lag in [1, 3, 6, 12]:
            ac = sub.autocorr(lag=lag)
            print(f"  {ac:>7.3f}", end="")
        print()

    full = regime_df["inflation"]
    print(f"  {'Full sample':<25s}", end="")
    for lag in [1, 3, 6, 12]:
        print(f"  {full.autocorr(lag=lag):>7.3f}", end="")
    print()

    ext_f = [f for f in EXTENDED_FEATURES if f in regime_df.columns]
    target = "inflation_12m_ahead"
    if target not in regime_df.columns:
        regime_df[target] = regime_df["inflation"].shift(-12)
    fc_df = regime_df[ext_f + [target]].dropna()

    periods = [
        ("Pre-Volcker", "1973-01", "1979-12"),
        ("Volcker disinflation", "1980-01", "1986-12"),
        ("Great Moderation I", "1987-01", "1999-12"),
        ("Great Moderation II", "2000-01", "2007-12"),
        ("Post-GFC", "2008-01", "2019-12"),
        ("COVID+", "2020-01", "2023-01"),
    ]

    print(f"\n  {'Period':<25s}  {'N':>5s}  {'Var(pi)':>8s}  {'AR R2':>8s}  {'Ridge R2':>9s}")
    print(f"  {'-'*60}")

    period_results = []
    for pname, ps, pe in periods:
        tr = fc_df.loc[:ps]; te = fc_df.loc[ps:pe]
        if len(tr) < 30 or len(te) < 6: continue
        var_pi = te[target].var()

        base_f = [f for f in BASE_FEATURES if f in fc_df.columns]
        sc = StandardScaler()
        Xtr = sc.fit_transform(tr[base_f]); Xte = sc.transform(te[base_f])
        m_ar = Ridge(alpha=1.0); m_ar.fit(Xtr, tr[target].values)
        r2_ar = r2_score(te[target].values, m_ar.predict(Xte))

        sc2 = StandardScaler()
        Xtr2 = sc2.fit_transform(tr[ext_f]); Xte2 = sc2.transform(te[ext_f])
        m_ridge = Ridge(alpha=10.0); m_ridge.fit(Xtr2, tr[target].values)
        r2_ridge = r2_score(te[target].values, m_ridge.predict(Xte2))

        period_results.append({"period": pname, "n": len(te), "var_pi": var_pi,
                               "r2_ar": r2_ar, "r2_ridge": r2_ridge})
        print(f"  {pname:<25s}  {len(te):>5d}  {var_pi:>8.2f}  {r2_ar:>+8.3f}  {r2_ridge:>+9.3f}")

    return period_results


# ================================================================
# VISUALIZATION
# ================================================================
def plot_results(all_results, period_results, output_path):
    model_colors = {"AR (lags only)": "#6b7280", "Pooled Ridge": C1,
                    "Ridge + regime": C3, "Ridge + regime prob": C4,
                    "Regime-conditional": C2, "RF pooled": C5}

    # ── Fig 1: MSE and R2 by horizon ──
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle("Inflation Forecasting: Does Regime Awareness Help?",
                 fontweight="bold", fontsize=14, y=1.02)

    model_names = list(all_results[HORIZONS[0]].keys())
    x = np.arange(len(HORIZONS)); w = 0.8 / len(model_names)

    for i, mname in enumerate(model_names):
        mses = [all_results[h].get(mname, {}).get("avg_mse", np.nan) for h in HORIZONS]
        color = model_colors.get(mname, f"C{i}")
        axes[0].bar(x + i*w, mses, w, label=mname, color=color, alpha=0.75,
                    edgecolor="white", lw=1)
    axes[0].set_xticks(x + w*(len(model_names)-1)/2)
    axes[0].set_xticklabels([f"{h}m" for h in HORIZONS])
    axes[0].set_ylabel("Test MSE (lower = better)")
    axes[0].set_title("Forecast MSE by horizon", fontweight="bold")
    axes[0].legend(frameon=True, fontsize=7)

    for i, mname in enumerate(model_names):
        r2s = [all_results[h].get(mname, {}).get("avg_r2", np.nan) for h in HORIZONS]
        color = model_colors.get(mname, f"C{i}")
        axes[1].bar(x + i*w, r2s, w, label=mname, color=color, alpha=0.75,
                    edgecolor="white", lw=1)
    axes[1].set_xticks(x + w*(len(model_names)-1)/2)
    axes[1].set_xticklabels([f"{h}m" for h in HORIZONS])
    axes[1].set_ylabel("Test R2"); axes[1].axhline(0, color="grey", lw=1, alpha=0.5)
    axes[1].set_title("Forecast R2 by horizon", fontweight="bold")
    axes[1].legend(frameon=True, fontsize=7)

    fig.tight_layout()
    path1 = f"{output_path}/forecast_regime_comparison.png"
    fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path1}")

    # ── Fig 2: Regime improvement ──
    fig, ax = plt.subplots(figsize=(12, 6))
    fig.suptitle("Regime Awareness: Improvement Over Pooled Ridge",
                 fontweight="bold", fontsize=14, y=1.02)

    regime_models = ["Ridge + regime", "Ridge + regime prob", "Regime-conditional"]
    x = np.arange(len(HORIZONS)); w = 0.8 / len(regime_models)
    for i, mname in enumerate(regime_models):
        deltas = []
        for h in HORIZONS:
            pooled = all_results[h].get("Pooled Ridge", {}).get("avg_mse", np.nan)
            regime = all_results[h].get(mname, {}).get("avg_mse", np.nan)
            delta_pct = (regime - pooled) / pooled * 100 if pooled > 0 else np.nan
            deltas.append(delta_pct)
        color = model_colors.get(mname, f"C{i}")
        ax.bar(x + i*w, deltas, w, label=mname, color=color, alpha=0.75,
               edgecolor="white", lw=1)
    ax.set_xticks(x + w*(len(regime_models)-1)/2)
    ax.set_xticklabels([f"{h}m" for h in HORIZONS])
    ax.axhline(0, color="grey", lw=1.5)
    ax.set_ylabel("% change in MSE vs Pooled Ridge (negative = better)")
    ax.set_title("Does conditioning on regimes reduce forecast error?", fontweight="bold")
    ax.legend(frameon=True, fontsize=9)

    fig.tight_layout()
    path2 = f"{output_path}/forecast_regime_improvement.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # ── Fig 3: Subsample stability ──
    if period_results:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle("Why 12-Month Inflation Forecasting Fails: Subsample Analysis",
                     fontweight="bold", fontsize=14, y=1.02)

        pnames = [p["period"] for p in period_results]
        r2_ars = [p["r2_ar"] for p in period_results]
        r2_ridges = [p["r2_ridge"] for p in period_results]
        var_pis = [p["var_pi"] for p in period_results]

        ax = axes[0]
        ax.scatter(var_pis, r2_ridges, s=80, color=C1, zorder=3, edgecolors="white", lw=1)
        for p, vp, r2 in zip(pnames, var_pis, r2_ridges):
            ax.annotate(p, (vp, r2), fontsize=8, textcoords="offset points", xytext=(5, 5))
        ax.axhline(0, color="grey", ls="--", lw=1)
        ax.set_xlabel("Inflation variance in test period")
        ax.set_ylabel("12-month forecast R2")
        ax.set_title("Forecast quality vs inflation variability", fontweight="bold")

        ax = axes[1]
        x_p = np.arange(len(pnames))
        ax.bar(x_p - 0.2, r2_ars, 0.35, label="AR", color="#6b7280", alpha=0.75, edgecolor="white")
        ax.bar(x_p + 0.2, r2_ridges, 0.35, label="Ridge", color=C1, alpha=0.75, edgecolor="white")
        ax.set_xticks(x_p); ax.set_xticklabels(pnames, rotation=30, ha="right", fontsize=9)
        ax.axhline(0, color="grey", ls="--", lw=1)
        ax.set_ylabel("12-month forecast R2")
        ax.set_title("R2 by test period", fontweight="bold")
        ax.legend(frameon=True, fontsize=9)

        fig.tight_layout()
        path3 = f"{output_path}/forecast_subsample_stability.png"
        fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path3}")
    else:
        path3 = None

    return [p for p in [path1, path2, path3] if p]


# ── Main ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    print("Fitting regimes...")
    regime_df, regime_names, n_regimes = get_regimes(df, n_regimes=2)
    print(f"  {n_regimes} regimes: {regime_names}")

    all_results = run_forecasting(regime_df, regime_names, n_regimes)
    period_results = subsample_stability(regime_df)
    paths = plot_results(all_results, period_results, OUTPUT_PATH)

    print(f"\nOutputs: {paths}")
    print("Done.")

Loading data...
Merged: 857 obs (1954-07 to 2025-12)
Fitting regimes...
  2 regimes: {0: 'Low inflation (2.4%)', 1: 'High inflation (7.4%)'}

REGIME-AWARE INFLATION FORECASTING (CBO NAIRU gap)

  === 1-month horizon (652 obs) ===

  Model                           MSE       MAE        R2
  -----------------------------------------------------
  AR (lags only)                0.450     0.459    +0.616
  Pooled Ridge                  0.568     0.560    +0.432
  Ridge + regime                0.499     0.516    +0.542
  Ridge + regime prob           0.490     0.511    +0.551
  Regime-conditional            0.518     0.512    +0.609
  RF pooled                     0.638     0.557    +0.511

  === 3-month horizon (650 obs) ===

  Model                           MSE       MAE        R2
  -----------------------------------------------------
  AR (lags only)                1.075     0.710    +0.041
  Pooled Ridge                  1.353     0.885    -0.534
  Ridge + regime                1.194  

In [19]:
#!/usr/bin/env python3
"""
Component 5: State-Dependent Local Projections
================================================
Jordà (2005) local projections with smooth regime interaction
following Auerbach & Gorodnichenko (2012).

Revision notes:
- Shock identification uses rich conditioning set (6 lags of inflation,
  unemp_gap, fed_funds, term_spread, capacity_gap, plus commodity price
  inflation if available) to purge predictable policy movements.
- Commodity price inflation partially addresses the price puzzle.
- LP controls match the shock conditioning set (Plagborg-Møller & Wolf 2021).
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import statsmodels.api as sm

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]


def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    # Commodity prices (PPI all commodities or similar)
    ppi = None
    for name in ["PPIACO.csv", "PPIFGS.csv", "PPIITM.csv"]:
        p = os.path.join(path, name)
        if os.path.exists(p):
            ppi = pd.read_csv(p, parse_dates=["observation_date"],
                              index_col="observation_date")
            print(f"  Loaded commodity prices from {name}")
            break
    if ppi is None:
        print("  No commodity price series found (PPIACO.csv etc), proceeding without")

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else:
        merged["expected_inflation"] = np.nan

    if ppi is not None:
        merged = merged.join(
            ppi.rename(columns={ppi.columns[0]: "ppi_commodities"}), how="left")
    else:
        merged["ppi_commodities"] = np.nan

    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["delta_pi"] = df["inflation"].diff()
    df["delta_u"] = df["unemployment"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])

    # Commodity price inflation (12-month)
    if "ppi_commodities" in df.columns and df["ppi_commodities"].notna().sum() > 24:
        df["commodity_inflation"] = df["ppi_commodities"].pct_change(12) * 100
    else:
        df["commodity_inflation"] = np.nan

    for var in ["inflation", "unemp_gap", "capacity_gap", "fed_funds",
                "term_spread", "fin_conditions", "commodity_inflation"]:
        for lag in [1, 2, 3, 4, 5, 6, 12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    for h in [1, 3, 6, 12]:
        df[f"inflation_{h}m_ahead"] = df["inflation"].shift(-h)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation", "L12_inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


def get_regimes(df, n_regimes=2):
    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())
    mod = sm.tsa.MarkovRegression(endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True)
    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf:
                best_llf = res.llf; best_res = res
        except:
            continue
    smoothed = best_res.smoothed_marginal_probabilities
    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0
    sorted_regimes = sorted(regime_means, key=regime_means.get)
    rdf = df.copy()
    for new_r in range(n_regimes):
        rdf[f"p_regime_{new_r}"] = smoothed[sorted_regimes[new_r]].values
    rdf["regime"] = np.argmax(
        np.column_stack([rdf[f"p_regime_{r}"].values for r in range(n_regimes)]), axis=1)
    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    labels = ["Low", "High"] if n_regimes == 2 else [f"R{r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}
    return rdf, regime_names, n_regimes


# ================================================================
# MONETARY POLICY SHOCK IDENTIFICATION
# ================================================================
def identify_mp_shocks(df):
    """
    Identify monetary policy shocks as residuals from a rich
    Taylor-type reaction function.

    Conditioning set: 6 lags of inflation, unemp_gap, fed_funds,
    term_spread, capacity_gap, plus commodity price inflation
    (if available). This is closer to the Fed's actual information
    set and partially addresses the price puzzle by controlling for
    cost-push shocks that both raise inflation and trigger rate hikes.

    References: Romer & Romer (2004), Coibion (2012), Ramey (2016)
    """
    print(f"\n{'='*70}")
    print("IDENTIFYING MONETARY POLICY SHOCKS")
    print(f"  Rich conditioning set (6 lags, commodity prices)")
    print(f"{'='*70}")

    # Build conditioning set
    base_vars = ["inflation", "unemp_gap", "fed_funds", "term_spread", "capacity_gap"]
    has_commodities = df["commodity_inflation"].notna().sum() > 100

    control_cols = []
    for var in base_vars:
        for lag in [1, 2, 3, 4, 5, 6]:
            col = f"L{lag}_{var}"
            if col in df.columns:
                control_cols.append(col)

    if has_commodities:
        for lag in [1, 2, 3, 4, 5, 6]:
            col = f"L{lag}_commodity_inflation"
            if col in df.columns:
                control_cols.append(col)
        print(f"  Including commodity price inflation (6 lags)")
    else:
        print(f"  No commodity prices available, using macro variables only")

    dep = df["delta_ff"].copy()
    controls = df[control_cols].copy()
    valid = dep.notna() & controls.notna().all(axis=1)
    dep = dep[valid]; controls = controls[valid]

    X = sm.add_constant(controls.values)
    model = sm.OLS(dep.values, X).fit(cov_type="HAC", cov_kwds={"maxlags": 12})

    print(f"\n  N = {model.nobs:.0f}, R2 = {model.rsquared:.3f}, adj-R2 = {model.rsquared_adj:.3f}")
    print(f"  Number of regressors: {len(control_cols)} + constant")

    # Print key coefficients (first lag of each variable)
    print(f"\n  Key coefficients (first lag only):")
    print(f"  {'Variable':<25s}  {'Coef':>8s}  {'SE':>8s}  {'t':>8s}")
    print(f"  {'-'*53}")
    param_names = ["const"] + control_cols
    for var in base_vars + (["commodity_inflation"] if has_commodities else []):
        col = f"L1_{var}"
        if col in param_names:
            idx = param_names.index(col)
            print(f"  {col:<25s}  {model.params[idx]:>8.4f}  {model.bse[idx]:>8.4f}  {model.tvalues[idx]:>8.2f}")

    # Store shocks
    shock = pd.Series(np.nan, index=df.index)
    shock[valid] = model.resid
    df = df.copy()
    df["mp_shock"] = shock

    # Standardise shock for interpretability (1 SD = 1 unit)
    shock_std = shock.std()
    df["mp_shock_std"] = shock / shock_std

    print(f"\n  Shock stats: mean={shock.mean():.4f}, sd={shock.std():.4f}")
    print(f"  Correlation with delta_ff: {shock.corr(dep):.3f}")

    # Diagnostic: does the shock still have the price puzzle?
    shock_valid = df["mp_shock"].dropna()
    corr_pi = shock_valid.corr(df.loc[shock_valid.index, "inflation"])
    corr_u = shock_valid.corr(df.loc[shock_valid.index, "unemp_gap"])
    print(f"  Correlation with inflation: {corr_pi:.3f}")
    print(f"  Correlation with unemp_gap: {corr_u:.3f}")

    return df, model


# ================================================================
# LOCAL PROJECTIONS
# ================================================================
def local_projections(df, shock_col="mp_shock", max_horizon=48, n_lags=6):
    """
    Jordà (2005) local projections with Auerbach-Gorodnichenko (2012)
    smooth regime interaction.

    Controls in the LP match the shock conditioning set
    (Plagborg-Møller & Wolf 2021).
    """
    print(f"\n{'='*70}")
    print("STATE-DEPENDENT LOCAL PROJECTIONS")
    print(f"  Shock variable: {shock_col}")
    print(f"  Max horizon: {max_horizon} months, {n_lags} control lags")
    print(f"{'='*70}")

    response_vars = ["inflation", "unemployment", "fed_funds", "capacity_util"]
    response_labels = ["Inflation (pp)", "Unemployment (pp)",
                       "Fed funds rate (pp)", "Capacity utilisation (pp)"]

    F_t = df["p_regime_1"].values
    shock = df[shock_col].values

    # Control variables matching the shock conditioning set
    control_cols = []
    for var in ["inflation", "unemployment", "fed_funds", "term_spread", "capacity_gap"]:
        for lag in range(1, n_lags + 1):
            col = f"_lp_L{lag}_{var}"
            df[col] = df[var].shift(lag)
            control_cols.append(col)

    if df["commodity_inflation"].notna().sum() > 100:
        for lag in range(1, n_lags + 1):
            col = f"_lp_L{lag}_commodity_inflation"
            df[col] = df["commodity_inflation"].shift(lag)
            control_cols.append(col)

    results = {}

    for var, label in zip(response_vars, response_labels):
        print(f"\n  --- Response: {label} ---")
        irf_high = np.full(max_horizon + 1, np.nan)
        irf_low = np.full(max_horizon + 1, np.nan)
        se_high = np.full(max_horizon + 1, np.nan)
        se_low = np.full(max_horizon + 1, np.nan)
        n_obs = np.full(max_horizon + 1, 0)

        irf_high[0] = 0.0; irf_low[0] = 0.0
        se_high[0] = 0.0; se_low[0] = 0.0

        for h in range(1, max_horizon + 1):
            y_ahead = df[var].shift(-h)
            y_base = df[var].shift(1)
            dep = y_ahead - y_base

            reg_df = pd.DataFrame({
                "dep": dep,
                "shock_H": shock * F_t,
                "shock_L": shock * (1 - F_t),
            }, index=df.index)

            for cc in control_cols:
                vals = df[cc].values if cc in df.columns else np.full(len(df), np.nan)
                reg_df[f"{cc}_H"] = vals * F_t
                reg_df[f"{cc}_L"] = vals * (1 - F_t)

            reg_df = reg_df.dropna()
            if len(reg_df) < 80:
                continue

            exog_cols = ["shock_H", "shock_L"]
            exog_cols += [c for c in reg_df.columns if c.startswith("_lp_L") and c.endswith("_H")]
            exog_cols += [c for c in reg_df.columns if c.startswith("_lp_L") and c.endswith("_L")]

            X = sm.add_constant(reg_df[exog_cols].values)
            y = reg_df["dep"].values

            try:
                model = sm.OLS(y, X).fit(
                    cov_type="HAC",
                    cov_kwds={"maxlags": max(h + 1, 12)}
                )
            except Exception:
                continue

            irf_high[h] = model.params[1]
            irf_low[h] = model.params[2]
            se_high[h] = model.bse[1]
            se_low[h] = model.bse[2]
            n_obs[h] = int(model.nobs)

        print(f"  {'Horizon':>8s}  {'IRF(High)':>10s}  {'SE(H)':>8s}  {'IRF(Low)':>10s}  "
              f"{'SE(L)':>8s}  {'Diff':>8s}  {'N':>5s}")
        print(f"  {'-'*65}")
        for h in [1, 3, 6, 12, 24, 36, 48]:
            if h <= max_horizon and not np.isnan(irf_high[h]):
                diff = irf_high[h] - irf_low[h]
                print(f"  {h:>8d}  {irf_high[h]:>10.4f}  {se_high[h]:>8.4f}  "
                      f"{irf_low[h]:>10.4f}  {se_low[h]:>8.4f}  {diff:>+8.4f}  {n_obs[h]:>5d}")

        results[var] = {
            "label": label,
            "irf_high": irf_high, "irf_low": irf_low,
            "se_high": se_high, "se_low": se_low,
            "n_obs": n_obs,
        }

    return results


# ================================================================
# LINEAR LOCAL PROJECTIONS (BASELINE)
# ================================================================
def linear_local_projections(df, shock_col="mp_shock", max_horizon=48, n_lags=6):
    """Standard (non-regime-dependent) local projections for comparison."""
    print(f"\n{'='*70}")
    print("LINEAR LOCAL PROJECTIONS (no regime interaction)")
    print(f"{'='*70}")

    response_vars = ["inflation", "unemployment", "fed_funds", "capacity_util"]
    response_labels = ["Inflation (pp)", "Unemployment (pp)",
                       "Fed funds rate (pp)", "Capacity utilisation (pp)"]

    shock = df[shock_col].values

    control_cols = []
    for var in ["inflation", "unemployment", "fed_funds", "term_spread", "capacity_gap"]:
        for lag in range(1, n_lags + 1):
            col = f"_lp_L{lag}_{var}"
            if col not in df.columns:
                df[col] = df[var].shift(lag)
            control_cols.append(col)
    if df["commodity_inflation"].notna().sum() > 100:
        for lag in range(1, n_lags + 1):
            col = f"_lp_L{lag}_commodity_inflation"
            if col not in df.columns:
                df[col] = df["commodity_inflation"].shift(lag)
            control_cols.append(col)

    results = {}
    for var, label in zip(response_vars, response_labels):
        irf = np.full(max_horizon + 1, np.nan)
        se = np.full(max_horizon + 1, np.nan)
        irf[0] = 0.0; se[0] = 0.0

        for h in range(1, max_horizon + 1):
            y_ahead = df[var].shift(-h)
            y_base = df[var].shift(1)
            dep = y_ahead - y_base

            reg_df = pd.DataFrame({"dep": dep, "shock": shock}, index=df.index)
            for cc in control_cols:
                reg_df[cc] = df[cc].values if cc in df.columns else np.nan
            reg_df = reg_df.dropna()
            if len(reg_df) < 80:
                continue

            exog_cols = ["shock"] + control_cols
            X = sm.add_constant(reg_df[exog_cols].values)
            y = reg_df["dep"].values

            try:
                model = sm.OLS(y, X).fit(
                    cov_type="HAC", cov_kwds={"maxlags": max(h + 1, 12)})
                irf[h] = model.params[1]
                se[h] = model.bse[1]
            except Exception:
                continue

        results[var] = {"label": label, "irf": irf, "se": se}
    return results


# ================================================================
# VISUALIZATION
# ================================================================
def plot_irfs(regime_results, linear_results, output_path):
    response_vars = ["inflation", "unemployment", "fed_funds", "capacity_util"]

    # ── Fig 1: Main IRF comparison (regime-dependent) ──
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle("Impulse Responses to a Monetary Policy Shock by Inflation Regime\n"
                 "(Jordà local projections, Newey-West HAC, 90% CI)",
                 fontweight="bold", fontsize=13, y=1.02)

    for ax, var in zip(axes.flat, response_vars):
        res = regime_results[var]
        horizons = np.arange(len(res["irf_high"]))

        h_irf = res["irf_high"]; h_se = res["se_high"]
        valid_h = ~np.isnan(h_irf)
        ax.plot(horizons[valid_h], h_irf[valid_h], lw=2, color=C2,
                label="High inflation regime")
        ax.fill_between(horizons[valid_h],
                        (h_irf - 1.65 * h_se)[valid_h],
                        (h_irf + 1.65 * h_se)[valid_h],
                        alpha=0.15, color=C2)

        l_irf = res["irf_low"]; l_se = res["se_low"]
        valid_l = ~np.isnan(l_irf)
        ax.plot(horizons[valid_l], l_irf[valid_l], lw=2, color=C1,
                label="Low inflation regime")
        ax.fill_between(horizons[valid_l],
                        (l_irf - 1.65 * l_se)[valid_l],
                        (l_irf + 1.65 * l_se)[valid_l],
                        alpha=0.15, color=C1)

        ax.axhline(0, color="grey", lw=1, ls="--")
        ax.set_xlabel("Months after shock")
        ax.set_ylabel(res["label"])
        ax.set_title(res["label"], fontweight="bold")
        ax.legend(frameon=True, fontsize=8)

    fig.tight_layout()
    path1 = f"{output_path}/lp_irfs_by_regime.png"
    fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path1}")

    # ── Fig 2: Regime difference ──
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle("Difference in Impulse Responses: High vs Low Inflation Regime\n"
                 "(positive = stronger response in high-inflation regime)",
                 fontweight="bold", fontsize=13, y=1.02)

    for ax, var in zip(axes.flat, response_vars):
        res = regime_results[var]
        horizons = np.arange(len(res["irf_high"]))
        diff = res["irf_high"] - res["irf_low"]
        se_diff = np.sqrt(res["se_high"]**2 + res["se_low"]**2)

        valid = ~np.isnan(diff)
        ax.plot(horizons[valid], diff[valid], lw=2, color=C3)
        ax.fill_between(horizons[valid],
                        (diff - 1.65 * se_diff)[valid],
                        (diff + 1.65 * se_diff)[valid],
                        alpha=0.2, color=C3)
        ax.axhline(0, color="grey", lw=1.5, ls="--")
        ax.set_xlabel("Months after shock")
        ax.set_ylabel("Difference (pp)")
        ax.set_title(res["label"], fontweight="bold")

        sig = valid.copy()
        sig[valid] = np.abs(diff[valid]) > 1.65 * se_diff[valid]
        if sig.any():
            for i in range(1, len(horizons)):
                if sig[i]:
                    ax.axvspan(horizons[i] - 0.5, horizons[i] + 0.5,
                               alpha=0.08, color=C4, zorder=0)

    fig.tight_layout()
    path2 = f"{output_path}/lp_irfs_regime_difference.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # ── Fig 3: Linear vs regime-dependent ──
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle("Monetary Policy Transmission: Linear vs Regime-Dependent IRFs",
                 fontweight="bold", fontsize=13, y=1.02)

    for ax, var in zip(axes.flat, response_vars):
        res_r = regime_results[var]
        res_l = linear_results[var]
        horizons = np.arange(len(res_r["irf_high"]))

        valid_lin = ~np.isnan(res_l["irf"])
        ax.plot(horizons[valid_lin], res_l["irf"][valid_lin], lw=2, color="#6b7280",
                label="Linear (pooled)", ls="--")
        ax.fill_between(horizons[valid_lin],
                        (res_l["irf"] - 1.65 * res_l["se"])[valid_lin],
                        (res_l["irf"] + 1.65 * res_l["se"])[valid_lin],
                        alpha=0.1, color="#6b7280")

        valid_h = ~np.isnan(res_r["irf_high"])
        ax.plot(horizons[valid_h], res_r["irf_high"][valid_h], lw=2, color=C2,
                label="High inflation")
        valid_l = ~np.isnan(res_r["irf_low"])
        ax.plot(horizons[valid_l], res_r["irf_low"][valid_l], lw=2, color=C1,
                label="Low inflation")

        ax.axhline(0, color="grey", lw=1, ls="--")
        ax.set_xlabel("Months after shock")
        ax.set_ylabel(res_r["label"])
        ax.set_title(res_r["label"], fontweight="bold")
        ax.legend(frameon=True, fontsize=8)

    fig.tight_layout()
    path3 = f"{output_path}/lp_irfs_linear_vs_regime.png"
    fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path3}")

    return [path1, path2, path3]


# ── Main ──
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    print("Fitting regimes...")
    regime_df, regime_names, n_regimes = get_regimes(df, n_regimes=2)
    print(f"  {n_regimes} regimes: {regime_names}")

    print("Identifying monetary policy shocks...")
    regime_df, shock_model = identify_mp_shocks(regime_df)

    print("Running state-dependent local projections...")
    regime_results = local_projections(regime_df, shock_col="mp_shock", max_horizon=48)

    print("Running linear local projections (baseline)...")
    linear_results = linear_local_projections(regime_df, shock_col="mp_shock", max_horizon=48)

    print("Plotting...")
    paths = plot_irfs(regime_results, linear_results, OUTPUT_PATH)

    print(f"\nOutputs: {paths}")
    print("Done.")

Loading data...
  No commodity price series found (PPIACO.csv etc), proceeding without
Merged: 857 obs (1954-07 to 2025-12)
Fitting regimes...
  2 regimes: {0: 'Low inflation (2.4%)', 1: 'High inflation (7.4%)'}
Identifying monetary policy shocks...

IDENTIFYING MONETARY POLICY SHOCKS
  Rich conditioning set (6 lags, commodity prices)
  No commodity prices available, using macro variables only

  N = 701, R2 = 0.340, adj-R2 = 0.310
  Number of regressors: 30 + constant

  Key coefficients (first lag only):
  Variable                       Coef        SE         t
  -----------------------------------------------------
  L1_inflation                -0.0074    0.0440     -0.17
  L1_unemp_gap                 0.0511    0.0753      0.68
  L1_fed_funds                 0.7957    0.1338      5.95
  L1_term_spread               0.5078    0.1995      2.55
  L1_capacity_gap              0.0999    0.0584      1.71

  Shock stats: mean=0.0000, sd=0.4199
  Correlation with delta_ff: 0.813
  Correlat

In [20]:
#!/usr/bin/env python3
"""
Component 6: Counterfactual Policy Simulation
===============================================
Estimates a reduced-form VAR on [inflation, unemployment, fed_funds],
then simulates counterfactual paths under mechanical Taylor rule adherence.

Revision notes:
- Constrained NLS Taylor rule (a_pi >= 0.5) so the rule actually
  responds to inflation, not just unemployment.
- Additional "textbook" Taylor rule variant (r*=2, a_pi=0.5, a_u=0.5)
  for a clean benchmark.
- Both counterfactuals shown side by side.
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from scipy.optimize import least_squares

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]


def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else:
        merged["expected_inflation"] = np.nan
    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation", "unemp_gap", "capacity_gap", "fed_funds", "term_spread", "fin_conditions"]:
        for lag in [1, 3, 6, 12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


# ================================================================
# TAYLOR RULE ESTIMATION
# ================================================================
def taylor_resid(params, y, pi, u_gap, ff_lag):
    rho, r_star, a_pi, a_u = params
    target = r_star + pi + a_pi * (pi - 2.0) + a_u * u_gap
    return y - (rho * ff_lag + (1 - rho) * target)


def estimate_taylor_rules(df):
    """
    Estimate Taylor rules:
    1. Constrained NLS (a_pi >= 0.5) so the rule actually responds to inflation
    2. Textbook Taylor (1993) with fixed parameters
    """
    print(f"\n{'='*70}")
    print("TAYLOR RULE ESTIMATION FOR COUNTERFACTUALS")
    print(f"{'='*70}")

    sub = df[["fed_funds", "ff_lag1", "inflation", "unemp_gap"]].dropna()
    y = sub["fed_funds"].values
    pi = sub["inflation"].values
    u_gap = sub["unemp_gap"].values
    ff_lag = sub["ff_lag1"].values

    # 1. Unconstrained (for reference)
    res_unc = least_squares(taylor_resid, [0.85, 2.0, 0.5, 0.5],
        args=(y, pi, u_gap, ff_lag),
        bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]))
    p_unc = res_unc.x

    # 2. Constrained: a_pi >= 0.5 (lower bound on inflation response)
    res_con = least_squares(taylor_resid, [0.85, 2.0, 0.5, 0.5],
        args=(y, pi, u_gap, ff_lag),
        bounds=([0, -5, 0.5, -5], [0.999, 10, 5, 5]))
    p_con = res_con.x

    # 3. Textbook Taylor (1993): r*=2, a_pi=0.5, a_u=0.5, no smoothing
    p_text = np.array([0.0, 2.0, 0.5, 0.5])

    # Print comparison
    print(f"\n  {'Specification':<25s}  {'rho':>6s}  {'r*':>6s}  {'a_pi':>6s}  {'a_u':>6s}  {'1+a_pi':>7s}  {'SSR':>10s}")
    print(f"  {'-'*75}")

    for label, params, res in [
        ("Unconstrained NLS", p_unc, res_unc),
        ("Constrained (a_pi>=0.5)", p_con, res_con),
    ]:
        ssr = np.sum(res.fun**2)
        tp = 1 + params[2]
        print(f"  {label:<25s}  {params[0]:>6.3f}  {params[1]:>6.3f}  {params[2]:>6.3f}  "
              f"{params[3]:>6.3f}  {tp:>7.2f}  {ssr:>10.1f}")

    # Textbook SSR
    ssr_text = np.sum(taylor_resid(p_text, y, pi, u_gap, ff_lag)**2)
    tp_text = 1 + p_text[2]
    print(f"  {'Textbook Taylor (1993)':<25s}  {p_text[0]:>6.3f}  {p_text[1]:>6.3f}  {p_text[2]:>6.3f}  "
          f"{p_text[3]:>6.3f}  {tp_text:>7.2f}  {ssr_text:>10.1f}")

    print(f"\n  Note: unconstrained a_pi = {p_unc[2]:.3f} < 0.5, indicating the data")
    print(f"  prefers a rule that barely responds to inflation. The constrained")
    print(f"  version forces a minimum inflation response consistent with theory.")

    return {
        "unconstrained": p_unc,
        "constrained": p_con,
        "textbook": p_text,
    }


def taylor_prescribed(pi, u_gap, params, with_smoothing=True, ff_prev=None):
    """Compute Taylor rule prescription."""
    rho, rstar, a_pi, a_u = params
    target = rstar + pi + a_pi * (pi - 2.0) + a_u * u_gap
    target = max(target, 0.0)  # ZLB

    if with_smoothing and rho > 0 and ff_prev is not None:
        return rho * ff_prev + (1 - rho) * target
    return target


# ================================================================
# VAR ESTIMATION
# ================================================================
def estimate_var(df, var_cols=["inflation", "unemployment", "fed_funds"], lags=6):
    print(f"\n{'='*70}")
    print(f"VAR ESTIMATION: {var_cols}, {lags} lags")
    print(f"{'='*70}")

    var_df = df[var_cols].dropna()
    model = VAR(var_df)
    lag_selection = model.select_order(maxlags=12)

    print(f"\n  Lag selection:")
    print(f"  {'Criterion':<10s}  {'Lags':>5s}")
    print(f"  {'-'*18}")
    for crit in ["aic", "bic", "hqic"]:
        val = getattr(lag_selection, crit)
        print(f"  {crit.upper():<10s}  {val:>5d}")

    bic_lags = lag_selection.bic
    use_lags = max(2, min(bic_lags if bic_lags is not None else lags, lags))
    print(f"\n  Using {use_lags} lags")

    result = model.fit(use_lags)
    print(f"  Observations: {result.nobs}")

    try:
        for i, col in enumerate(var_cols):
            ss_res = np.sum(result.resid.iloc[:, i].values ** 2)
            y = var_df[col].values[use_lags:]
            ss_tot = np.sum((y - y.mean()) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
            print(f"\n  {col}: R2 = {r2:.3f}")
    except Exception:
        print("\n  (R2 diagnostics unavailable)")

    return result, var_df


# ================================================================
# COUNTERFACTUAL SIMULATION
# ================================================================
def run_counterfactual(var_result, df, taylor_params, rule_label,
                        var_cols=["inflation", "unemployment", "fed_funds"],
                        use_smoothing=True):
    """
    Simulate counterfactual path where the Fed follows the given
    Taylor rule, using VAR dynamics for inflation and unemployment.
    """
    n_lags = var_result.k_ar
    coefs = var_result.coefs
    intercept = var_result.coefs_exog[:, 0] if var_result.coefs_exog.shape[1] > 0 else np.zeros(len(var_cols))

    pi_idx = var_cols.index("inflation")
    u_idx = var_cols.index("unemployment")
    ff_idx = var_cols.index("fed_funds")

    var_df = df[var_cols].dropna()
    nrou_series = df["nrou"].reindex(var_df.index)

    cf_data = var_df.values.copy()
    start = n_lags + 12

    for t in range(start, len(var_df)):
        lag_vals = np.zeros((n_lags, len(var_cols)))
        for lag in range(n_lags):
            lag_vals[lag] = cf_data[t - lag - 1]

        pred = intercept.copy()
        for lag in range(n_lags):
            pred += coefs[lag] @ lag_vals[lag]

        cf_pi = cf_data[t - 1, pi_idx]
        cf_u = cf_data[t - 1, u_idx]
        cf_nrou = nrou_series.iloc[t] if t < len(nrou_series) and not np.isnan(nrou_series.iloc[t]) else 5.0
        cf_u_gap = cf_u - cf_nrou
        cf_ff_prev = cf_data[t - 1, ff_idx]

        taylor_ff = taylor_prescribed(
            cf_pi, cf_u_gap, taylor_params,
            with_smoothing=use_smoothing, ff_prev=cf_ff_prev)

        cf_data[t, pi_idx] = pred[pi_idx]
        cf_data[t, u_idx] = pred[u_idx]
        cf_data[t, ff_idx] = taylor_ff

    result_df = pd.DataFrame({
        "actual_inflation": var_df["inflation"].values,
        f"cf_inflation_{rule_label}": cf_data[:, pi_idx],
        "actual_unemployment": var_df["unemployment"].values,
        f"cf_unemployment_{rule_label}": cf_data[:, u_idx],
        "actual_ff": var_df["fed_funds"].values,
        f"cf_ff_{rule_label}": cf_data[:, ff_idx],
    }, index=var_df.index)

    return result_df.iloc[start:]


def episode_counterfactuals(var_result, df, taylor_rules, episodes,
                             var_cols=["inflation", "unemployment", "fed_funds"]):
    """Run counterfactuals for specific episodes, comparing rule specifications."""
    print(f"\n{'='*70}")
    print("EPISODE COUNTERFACTUALS")
    print(f"{'='*70}")

    n_lags = var_result.k_ar
    coefs = var_result.coefs
    intercept = var_result.coefs_exog[:, 0] if var_result.coefs_exog.shape[1] > 0 else np.zeros(len(var_cols))

    pi_idx = var_cols.index("inflation")
    u_idx = var_cols.index("unemployment")
    ff_idx = var_cols.index("fed_funds")

    all_episodes = {}

    for ep_name, ep_start, ep_end in episodes:
        print(f"\n  --- {ep_name} ---")
        var_df = df[var_cols].dropna()
        nrou_series = df["nrou"].reindex(var_df.index)

        start_dt = pd.Timestamp(ep_start)
        end_dt = pd.Timestamp(ep_end)
        valid_dates = var_df.index[(var_df.index >= start_dt) & (var_df.index <= end_dt)]
        if len(valid_dates) == 0:
            print(f"    No data, skipping")
            continue
        start_idx = var_df.index.get_loc(valid_dates[0])
        end_idx = var_df.index.get_loc(valid_dates[-1])
        if start_idx < n_lags or end_idx - start_idx < 6:
            print(f"    Too few observations, skipping")
            continue

        ep_data = {"dates": var_df.index[start_idx:end_idx + 1],
                   "actual": var_df.values[start_idx:end_idx + 1]}

        print(f"  {'':>22s}  {'Actual':>8s}", end="")
        for rl in taylor_rules:
            print(f"  {rl[:10]:>10s}", end="")
        print()
        print(f"  {'-'*55}")

        for rule_label, rule_params in taylor_rules.items():
            cf_data = var_df.values.copy()
            use_smoothing = rule_label != "textbook"  # textbook has rho=0

            for t in range(start_idx, end_idx + 1):
                lag_vals = np.zeros((n_lags, len(var_cols)))
                for lag in range(n_lags):
                    lag_vals[lag] = cf_data[t - lag - 1]

                pred = intercept.copy()
                for lag in range(n_lags):
                    pred += coefs[lag] @ lag_vals[lag]

                cf_pi = cf_data[t - 1, pi_idx]
                cf_u = cf_data[t - 1, u_idx]
                cf_nrou = nrou_series.iloc[t] if not np.isnan(nrou_series.iloc[t]) else 5.0
                cf_u_gap = cf_u - cf_nrou
                cf_ff_prev = cf_data[t - 1, ff_idx]

                taylor_ff = taylor_prescribed(
                    cf_pi, cf_u_gap, rule_params,
                    with_smoothing=use_smoothing, ff_prev=cf_ff_prev)

                cf_data[t, pi_idx] = pred[pi_idx]
                cf_data[t, u_idx] = pred[u_idx]
                cf_data[t, ff_idx] = taylor_ff

            ep_data[rule_label] = cf_data[start_idx:end_idx + 1]

        # Print summary
        actual = ep_data["actual"]
        for var_name, idx, label in [("Fed funds", ff_idx, "ff"),
                                      ("Inflation", pi_idx, "pi"),
                                      ("Unemployment", u_idx, "u")]:
            line = f"  {var_name + ' (mean)':>22s}  {actual[:, idx].mean():>8.2f}"
            for rl in taylor_rules:
                cf = ep_data[rl]
                line += f"  {cf[:, idx].mean():>10.2f}"
            print(line)

        all_episodes[ep_name] = ep_data

    return all_episodes


# ================================================================
# VISUALIZATION
# ================================================================
def plot_results(full_cf_constrained, full_cf_textbook, all_episodes,
                 taylor_rules, output_path):

    # ── Fig 1: Full-sample counterfactual (constrained rule) ──
    cf = full_cf_constrained
    fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)
    fig.suptitle("Counterfactual: What If the Fed Had Followed a Taylor Rule?\n"
                 "(Constrained NLS: a_pi >= 0.5, with interest rate smoothing)",
                 fontweight="bold", fontsize=13, y=0.995)

    panels = [
        ("actual_ff", "cf_ff_constrained", "Fed funds rate (%)", C1),
        ("actual_inflation", "cf_inflation_constrained", "Inflation (%)", C2),
        ("actual_unemployment", "cf_unemployment_constrained", "Unemployment (%)", C3),
    ]

    for ax, (act_col, cf_col, ylabel, color) in zip(axes, panels):
        shade_recessions(ax)
        ax.plot(cf.index, cf[act_col], lw=1.8, color=color,
                label="Actual", zorder=3)
        ax.plot(cf.index, cf[cf_col], lw=1.5, color=color,
                alpha=0.5, ls="--", label="Counterfactual", zorder=2)
        ax.set_ylabel(ylabel)
        ax.set_title(ylabel, fontweight="bold")
        ax.legend(frameon=True, fontsize=8, loc="upper right")
        if "inflation" in act_col and "ff" not in act_col:
            ax.axhline(2, color="grey", ls=":", lw=1, alpha=0.5)

    axes[-1].set_xlabel("Year")
    fig.tight_layout()
    path1 = f"{output_path}/counterfactual_full_sample.png"
    fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path1}")

    # ── Fig 2: Constrained vs textbook comparison (gap) ──
    fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)
    fig.suptitle("Counterfactual Comparison: Constrained NLS vs Textbook Taylor Rule",
                 fontweight="bold", fontsize=13, y=0.995)

    cf_t = full_cf_textbook
    # Align indices
    common_idx = cf.index.intersection(cf_t.index)

    ax = axes[0]; shade_recessions(ax)
    ax.plot(common_idx, cf.loc[common_idx, "actual_ff"], lw=1.8, color="#374151",
            label="Actual")
    ax.plot(common_idx, cf.loc[common_idx, "cf_ff_constrained"], lw=1.5, color=C1,
            ls="--", label="Constrained NLS")
    ax.plot(common_idx, cf_t.loc[common_idx, "cf_ff_textbook"], lw=1.5, color=C4,
            ls="--", label="Textbook (r*=2, a_pi=0.5, a_u=0.5)")
    ax.set_ylabel("Fed funds rate (%)"); ax.set_title("Fed funds rate", fontweight="bold")
    ax.legend(frameon=True, fontsize=8)

    ax = axes[1]; shade_recessions(ax)
    ax.plot(common_idx, cf.loc[common_idx, "actual_inflation"], lw=1.8, color="#374151",
            label="Actual")
    ax.plot(common_idx, cf.loc[common_idx, "cf_inflation_constrained"], lw=1.5, color=C1,
            ls="--", label="Constrained NLS")
    ax.plot(common_idx, cf_t.loc[common_idx, "cf_inflation_textbook"], lw=1.5, color=C4,
            ls="--", label="Textbook")
    ax.axhline(2, color="grey", ls=":", lw=1, alpha=0.5)
    ax.set_ylabel("Inflation (%)"); ax.set_title("Inflation", fontweight="bold")
    ax.legend(frameon=True, fontsize=8)

    ax = axes[2]; shade_recessions(ax)
    ax.plot(common_idx, cf.loc[common_idx, "actual_unemployment"], lw=1.8, color="#374151",
            label="Actual")
    ax.plot(common_idx, cf.loc[common_idx, "cf_unemployment_constrained"], lw=1.5, color=C1,
            ls="--", label="Constrained NLS")
    ax.plot(common_idx, cf_t.loc[common_idx, "cf_unemployment_textbook"], lw=1.5, color=C4,
            ls="--", label="Textbook")
    ax.set_ylabel("Unemployment (%)"); ax.set_xlabel("Year")
    ax.set_title("Unemployment", fontweight="bold")
    ax.legend(frameon=True, fontsize=8)

    fig.tight_layout()
    path2 = f"{output_path}/counterfactual_rule_comparison.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # ── Fig 3: Episode panels ──
    n_eps = len(all_episodes)
    if n_eps == 0:
        return [path1, path2]

    fig, axes = plt.subplots(n_eps, 2, figsize=(16, 4 * n_eps))
    if n_eps == 1:
        axes = axes.reshape(1, -1)
    fig.suptitle("Counterfactual Paths by Historical Episode",
                 fontweight="bold", fontsize=14, y=1.01)

    rule_colors = {"constrained": C1, "textbook": C4}

    for row, (ep_name, ep_data) in enumerate(all_episodes.items()):
        dates = ep_data["dates"]
        actual = ep_data["actual"]
        pi_idx, ff_idx = 0, 2  # inflation=0, fed_funds=2

        # Left: fed funds
        ax = axes[row, 0]
        ax.plot(dates, actual[:, ff_idx], lw=2, color="#374151", label="Actual")
        for rl in taylor_rules:
            if rl in ep_data:
                color = rule_colors.get(rl, C5)
                ax.plot(dates, ep_data[rl][:, ff_idx], lw=1.5, color=color,
                        ls="--", label=f"{rl}")
        ax.set_ylabel("Fed funds (%)")
        ax.set_title(f"{ep_name}: Fed funds", fontweight="bold", fontsize=10)
        ax.legend(frameon=True, fontsize=7, loc="best")

        # Right: inflation
        ax = axes[row, 1]
        ax.plot(dates, actual[:, pi_idx], lw=2, color="#374151", label="Actual")
        for rl in taylor_rules:
            if rl in ep_data:
                color = rule_colors.get(rl, C5)
                ax.plot(dates, ep_data[rl][:, pi_idx], lw=1.5, color=color,
                        ls="--", label=f"{rl}")
        ax.axhline(2, color="grey", ls=":", lw=1, alpha=0.5)
        ax.set_ylabel("Inflation (%)")
        ax.set_title(f"{ep_name}: Inflation", fontweight="bold", fontsize=10)
        ax.legend(frameon=True, fontsize=7, loc="best")

    fig.tight_layout()
    path3 = f"{output_path}/counterfactual_episodes.png"
    fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path3}")

    return [path1, path2, path3]


# ── Main ──
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    print("Estimating Taylor rules...")
    taylor_rules = estimate_taylor_rules(df)

    print("Estimating VAR...")
    var_result, var_df = estimate_var(df, lags=6)

    # Full-sample counterfactuals
    print("\nRunning full-sample counterfactual (constrained rule)...")
    full_cf_con = run_counterfactual(var_result, df, taylor_rules["constrained"],
                                      "constrained", use_smoothing=True)
    print(f"  Mean actual ff: {full_cf_con['actual_ff'].mean():.2f}")
    print(f"  Mean CF ff: {full_cf_con['cf_ff_constrained'].mean():.2f}")
    print(f"  Mean actual inflation: {full_cf_con['actual_inflation'].mean():.2f}")
    print(f"  Mean CF inflation: {full_cf_con['cf_inflation_constrained'].mean():.2f}")

    print("\nRunning full-sample counterfactual (textbook rule)...")
    full_cf_text = run_counterfactual(var_result, df, taylor_rules["textbook"],
                                       "textbook", use_smoothing=False)
    print(f"  Mean CF ff (textbook): {full_cf_text['cf_ff_textbook'].mean():.2f}")
    print(f"  Mean CF inflation (textbook): {full_cf_text['cf_inflation_textbook'].mean():.2f}")

    # Episode counterfactuals
    EPISODES = [
        ("Burns accommodation (1971-78)",     "1971-01", "1978-06"),
        ("Volcker tightening (1979-82)",       "1979-01", "1983-12"),
        ("Greenspan era (1987-2000)",          "1987-01", "2000-12"),
        ("Post-GFC ZLB (2009-15)",             "2009-01", "2015-12"),
        ("COVID and after (2020-23)",          "2020-01", "2023-06"),
    ]

    all_episodes = episode_counterfactuals(
        var_result, df,
        {"constrained": taylor_rules["constrained"],
         "textbook": taylor_rules["textbook"]},
        EPISODES)

    print("\nPlotting...")
    paths = plot_results(full_cf_con, full_cf_text, all_episodes,
                         {"constrained": taylor_rules["constrained"],
                          "textbook": taylor_rules["textbook"]},
                         OUTPUT_PATH)

    print(f"\nOutputs: {paths}")
    print("Done.")

Loading data...
Merged: 857 obs (1954-07 to 2025-12)
Estimating Taylor rules...

TAYLOR RULE ESTIMATION FOR COUNTERFACTUALS

  Specification                 rho      r*    a_pi     a_u   1+a_pi         SSR
  ---------------------------------------------------------------------------
  Unconstrained NLS           0.974   1.945  -0.086  -1.609     0.91       191.4
  Constrained (a_pi>=0.5)     0.982   1.240   0.500  -2.098     1.50       192.4
  Textbook Taylor (1993)      0.000   2.000   0.500   0.500     1.50     11959.7

  Note: unconstrained a_pi = -0.086 < 0.5, indicating the data
  prefers a rule that barely responds to inflation. The constrained
  version forces a minimum inflation response consistent with theory.
Estimating VAR...

VAR ESTIMATION: ['inflation', 'unemployment', 'fed_funds'], 6 lags

  Lag selection:
  Criterion    Lags
  ------------------
  AIC            11
  BIC             2
  HQIC            2

  Using 2 lags
  Observations: 843

  inflation: R2 = 0.984

  un

In [18]:
#!/usr/bin/env python3
"""
Component 7: Regime-Dependent Structural VAR
=============================================
Estimates a 3-variable SVAR [inflation, unemployment_gap, fed_funds]
with Cholesky identification, separately for each inflation regime.

Cholesky ordering: inflation -> unemp_gap -> fed_funds
(monetary policy responds contemporaneously to both, but neither
responds within the month to a policy shock)

Produces regime-specific impulse response functions with
bootstrapped confidence bands.
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})


def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"],
                                   index_col="observation_date")
        raise FileNotFoundError(f"None of {names} found in {path}")

    cpi = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu = _read(["TCU.csv"])
    gs10 = _read(["GS10.csv"])
    nfci_w = _read(["NFCI.csv"]); nfci = nfci_w.resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate(method="linear")
    try:
        mich = _read(["MICH.csv"])
    except FileNotFoundError:
        mich = None

    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi", "unemployment", "fed_funds", "nrou"])

    if mich is not None:
        merged = merged.join(
            mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else:
        merged["expected_inflation"] = np.nan
    print(f"Merged: {len(merged)} obs ({merged.index.min():%Y-%m} to {merged.index.max():%Y-%m})")
    return merged


def engineer_features(data):
    df = data.copy()
    df["inflation"] = df["cpi"].pct_change(12) * 100
    df["unemp_gap"] = df["unemployment"] - df["nrou"]
    df["capacity_gap"] = df["capacity_util"] - 100.0
    df["term_spread"] = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"] = df["fed_funds"] - df["inflation"]
    df["delta_ff"] = df["fed_funds"].diff()
    df["ff_lag1"] = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"] = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation", "unemp_gap", "capacity_gap", "fed_funds", "term_spread", "fin_conditions"]:
        for lag in [1, 3, 6, 12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df


RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]


def get_regimes(df, n_regimes=2):
    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())
    mod = sm.tsa.MarkovRegression(endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True)
    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf:
                best_llf = res.llf; best_res = res
        except:
            continue
    smoothed = best_res.smoothed_marginal_probabilities
    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0
    sorted_regimes = sorted(regime_means, key=regime_means.get)
    rdf = df.copy()
    for new_r in range(n_regimes):
        rdf[f"p_regime_{new_r}"] = smoothed[sorted_regimes[new_r]].values
    rdf["regime"] = np.argmax(
        np.column_stack([rdf[f"p_regime_{r}"].values for r in range(n_regimes)]), axis=1)
    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    labels = ["Low", "High"] if n_regimes == 2 else [f"R{r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}
    return rdf, regime_names, n_regimes


# ================================================================
# SVAR ESTIMATION
# ================================================================
def estimate_svar(df_sub, var_cols, n_lags, label=""):
    """
    Estimate VAR and compute Cholesky-identified IRFs.
    Returns VAR result and IRFs.
    """
    sub = df_sub[var_cols].dropna()
    if len(sub) < n_lags * len(var_cols) + 30:
        print(f"  [{label}] Too few observations ({len(sub)}), skipping")
        return None, None

    model = VAR(sub)
    result = model.fit(n_lags)

    # Compute IRFs with Cholesky decomposition (default in statsmodels)
    irf = result.irf(periods=48)

    print(f"  [{label}] N={result.nobs}, lags={n_lags}, "
          f"AIC={result.aic:.1f}, BIC={result.bic:.1f}")

    return result, irf


def bootstrap_irf_ci(df_sub, var_cols, n_lags, n_boot=500, periods=48,
                      ci_level=0.90):
    """
    Bootstrap confidence intervals for Cholesky IRFs.
    Uses residual resampling (standard VAR bootstrap).
    """
    sub = df_sub[var_cols].dropna()
    n_vars = len(var_cols)

    # Estimate base model
    model = VAR(sub)
    result = model.fit(n_lags)
    base_irf = result.irf(periods=periods).irfs  # (periods+1, n_vars, n_vars)

    residuals = result.resid  # (T-p, n_vars)
    fitted = result.fittedvalues
    T = len(residuals)

    boot_irfs = np.zeros((n_boot, periods + 1, n_vars, n_vars))
    successful = 0

    for b in range(n_boot):
        # Resample residuals
        boot_idx = np.random.randint(0, T, size=T)
        boot_resid = residuals.values[boot_idx]

        # Reconstruct data
        boot_y = fitted.values + boot_resid
        boot_df = pd.DataFrame(boot_y, columns=var_cols,
                                index=fitted.index)

        try:
            boot_model = VAR(boot_df)
            boot_result = boot_model.fit(n_lags)
            boot_irf_obj = boot_result.irf(periods=periods)
            boot_irfs[successful] = boot_irf_obj.irfs
            successful += 1
        except Exception:
            continue

    if successful < 50:
        print(f"    Bootstrap: only {successful}/{n_boot} successful, CI may be unreliable")
        return None

    boot_irfs = boot_irfs[:successful]
    alpha = (1 - ci_level) / 2
    ci_lower = np.quantile(boot_irfs, alpha, axis=0)
    ci_upper = np.quantile(boot_irfs, 1 - alpha, axis=0)

    print(f"    Bootstrap: {successful}/{n_boot} successful replications")

    return {"lower": ci_lower, "upper": ci_upper, "median": np.median(boot_irfs, axis=0)}


# ================================================================
# REGIME-DEPENDENT ANALYSIS
# ================================================================
def regime_svar_analysis(regime_df, regime_names, n_regimes):
    """
    Estimate SVAR separately for each regime and compare IRFs.
    """
    print(f"\n{'='*70}")
    print("REGIME-DEPENDENT STRUCTURAL VAR")
    print(f"  Ordering: inflation -> unemp_gap -> fed_funds (Cholesky)")
    print(f"{'='*70}")

    var_cols = ["inflation", "unemp_gap", "fed_funds"]
    n_lags = 6

    # Full-sample SVAR
    print(f"\n  --- Full sample ---")
    full_result, full_irf = estimate_svar(regime_df, var_cols, n_lags, "Full sample")

    # Full-sample bootstrap CI
    print(f"    Computing bootstrap confidence intervals...")
    full_ci = bootstrap_irf_ci(regime_df, var_cols, n_lags, n_boot=500, periods=48)

    # Regime-specific SVARs
    regime_results = {}
    regime_irfs = {}
    regime_cis = {}

    for r in range(n_regimes):
        label = regime_names[r]
        print(f"\n  --- {label} ---")
        rsub = regime_df[regime_df["regime"] == r].copy()

        result, irf = estimate_svar(rsub, var_cols, n_lags, label)
        if result is None:
            continue

        regime_results[r] = result
        regime_irfs[r] = irf

        print(f"    Computing bootstrap confidence intervals...")
        ci = bootstrap_irf_ci(rsub, var_cols, n_lags, n_boot=500, periods=48)
        regime_cis[r] = ci

    # Print key IRF comparisons
    print(f"\n  === Monetary policy shock -> Inflation (cumulative) ===")
    print(f"  {'Horizon':>8s}", end="")
    print(f"  {'Full':>10s}", end="")
    for r in sorted(regime_results.keys()):
        print(f"  {regime_names[r][:15]:>15s}", end="")
    print()
    print(f"  {'-'*50}")

    shock_idx = var_cols.index("fed_funds")
    resp_idx = var_cols.index("inflation")

    for h in [1, 3, 6, 12, 24, 36, 48]:
        print(f"  {h:>8d}", end="")
        if full_irf is not None:
            print(f"  {full_irf.irfs[h, resp_idx, shock_idx]:>10.4f}", end="")
        else:
            print(f"  {'N/A':>10s}", end="")
        for r in sorted(regime_results.keys()):
            irf_val = regime_irfs[r].irfs[h, resp_idx, shock_idx]
            print(f"  {irf_val:>15.4f}", end="")
        print()

    print(f"\n  === Monetary policy shock -> Unemployment gap ===")
    resp_idx_u = var_cols.index("unemp_gap")
    print(f"  {'Horizon':>8s}", end="")
    print(f"  {'Full':>10s}", end="")
    for r in sorted(regime_results.keys()):
        print(f"  {regime_names[r][:15]:>15s}", end="")
    print()
    print(f"  {'-'*50}")

    for h in [1, 3, 6, 12, 24, 36, 48]:
        print(f"  {h:>8d}", end="")
        if full_irf is not None:
            print(f"  {full_irf.irfs[h, resp_idx_u, shock_idx]:>10.4f}", end="")
        else:
            print(f"  {'N/A':>10s}", end="")
        for r in sorted(regime_results.keys()):
            irf_val = regime_irfs[r].irfs[h, resp_idx_u, shock_idx]
            print(f"  {irf_val:>15.4f}", end="")
        print()

    # Forecast error variance decomposition
    print(f"\n  === Forecast Error Variance Decomposition (inflation) ===")
    if full_result is not None:
        fevd_obj = full_result.fevd(48)
        fevd = fevd_obj.decomp  # shape: (n_vars, periods, n_vars)
        # fevd[i, h, j] = fraction of variance of variable i at horizon h from shock j
        print(f"  {'Horizon':>8s}  {'pi shock':>10s}  {'u_gap shock':>12s}  {'ff shock':>10s}")
        print(f"  {'-'*44}")
        for h in [0, 2, 5, 11, 23, 47]:
            pi_pi = fevd[resp_idx, h, 0]
            pi_u = fevd[resp_idx, h, 1]
            pi_ff = fevd[resp_idx, h, 2]
            print(f"  {h+1:>8d}  {pi_pi:>10.3f}  {pi_u:>12.3f}  {pi_ff:>10.3f}")

    return {
        "full": {"result": full_result, "irf": full_irf, "ci": full_ci},
        "regimes": {r: {"result": regime_results[r], "irf": regime_irfs[r],
                        "ci": regime_cis.get(r)} for r in regime_results},
        "var_cols": var_cols,
        "regime_names": regime_names,
    }


# ================================================================
# VISUALIZATION
# ================================================================
def plot_svar_results(svar_results, output_path):
    """Plot SVAR impulse response functions."""
    var_cols = svar_results["var_cols"]
    regime_names = svar_results["regime_names"]
    regime_colors = [C5, C2, C4]  # low, high

    shock_idx = var_cols.index("fed_funds")
    response_labels = {
        "inflation": "Inflation (pp)",
        "unemp_gap": "Unemployment gap (pp)",
        "fed_funds": "Fed funds (pp)",
    }

    # ── Fig 1: Full-sample IRFs to monetary policy shock ──
    full_irf = svar_results["full"]["irf"]
    full_ci = svar_results["full"]["ci"]

    if full_irf is not None:
        fig, axes = plt.subplots(1, 3, figsize=(16, 5))
        fig.suptitle("Structural IRFs to a Monetary Policy Shock (Cholesky identification)\n"
                     "Full sample, 90% bootstrap confidence bands",
                     fontweight="bold", fontsize=13, y=1.05)

        horizons = np.arange(full_irf.irfs.shape[0])

        for ax, (resp_idx, var_name) in zip(axes, enumerate(var_cols)):
            resp_idx_val = resp_idx
            irf_vals = full_irf.irfs[:, resp_idx_val, shock_idx]
            ax.plot(horizons, irf_vals, lw=2, color=C1)

            if full_ci is not None:
                ax.fill_between(horizons,
                                full_ci["lower"][:, resp_idx_val, shock_idx],
                                full_ci["upper"][:, resp_idx_val, shock_idx],
                                alpha=0.2, color=C1)

            ax.axhline(0, color="grey", lw=1, ls="--")
            ax.set_xlabel("Months")
            ax.set_ylabel(response_labels[var_name])
            ax.set_title(f"Response of {response_labels[var_name]}", fontweight="bold", fontsize=10)

        fig.tight_layout()
        path1 = f"{output_path}/svar_irfs_full_sample.png"
        fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path1}")
    else:
        path1 = None

    # ── Fig 2: Regime-dependent IRFs (main result) ──
    regime_data = svar_results["regimes"]
    if len(regime_data) >= 2:
        fig, axes = plt.subplots(2, 3, figsize=(16, 10))
        fig.suptitle("Monetary Policy Transmission by Inflation Regime\n"
                     "(Cholesky SVAR, 90% bootstrap CI)",
                     fontweight="bold", fontsize=13, y=1.02)

        # Top row: IRFs by regime
        for col, (resp_idx_val, var_name) in enumerate(zip(range(len(var_cols)), var_cols)):
            ax = axes[0, col]

            for r in sorted(regime_data.keys()):
                irf_obj = regime_data[r]["irf"]
                ci = regime_data[r].get("ci")
                color = regime_colors[r]
                label = regime_names[r]

                horizons = np.arange(irf_obj.irfs.shape[0])
                irf_vals = irf_obj.irfs[:, resp_idx_val, shock_idx]
                ax.plot(horizons, irf_vals, lw=2, color=color, label=label)

                if ci is not None:
                    ax.fill_between(horizons,
                                    ci["lower"][:, resp_idx_val, shock_idx],
                                    ci["upper"][:, resp_idx_val, shock_idx],
                                    alpha=0.12, color=color)

            ax.axhline(0, color="grey", lw=1, ls="--")
            ax.set_xlabel("Months")
            ax.set_ylabel(response_labels[var_name])
            ax.set_title(f"Response of {response_labels[var_name]}", fontweight="bold", fontsize=10)
            ax.legend(frameon=True, fontsize=7)

        # Bottom row: difference (high minus low)
        r_keys = sorted(regime_data.keys())
        if len(r_keys) >= 2:
            r_low, r_high = r_keys[0], r_keys[1]
            for col, (resp_idx_val, var_name) in enumerate(zip(range(len(var_cols)), var_cols)):
                ax = axes[1, col]

                irf_low = regime_data[r_low]["irf"].irfs[:, resp_idx_val, shock_idx]
                irf_high = regime_data[r_high]["irf"].irfs[:, resp_idx_val, shock_idx]
                diff = irf_high - irf_low
                horizons = np.arange(len(diff))

                ax.plot(horizons, diff, lw=2, color=C3)
                ax.axhline(0, color="grey", lw=1.5, ls="--")
                ax.set_xlabel("Months")
                ax.set_ylabel("Difference (pp)")
                ax.set_title(f"Difference: high minus low regime", fontweight="bold", fontsize=10)

                # Approximate CI on difference using bootstrap
                ci_low = regime_data[r_low].get("ci")
                ci_high = regime_data[r_high].get("ci")
                if ci_low is not None and ci_high is not None:
                    # Conservative: sum of variances
                    se_low = (ci_low["upper"][:, resp_idx_val, shock_idx] -
                              ci_low["lower"][:, resp_idx_val, shock_idx]) / (2 * 1.645)
                    se_high = (ci_high["upper"][:, resp_idx_val, shock_idx] -
                               ci_high["lower"][:, resp_idx_val, shock_idx]) / (2 * 1.645)
                    se_diff = np.sqrt(se_low**2 + se_high**2)
                    ax.fill_between(horizons, diff - 1.645 * se_diff,
                                    diff + 1.645 * se_diff, alpha=0.2, color=C3)

        fig.tight_layout()
        path2 = f"{output_path}/svar_irfs_by_regime.png"
        fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path2}")
    else:
        path2 = None

    # ── Fig 3: FEVD comparison ──
    if full_irf is not None and len(regime_data) >= 2:
        fig, axes = plt.subplots(1, 3, figsize=(16, 5))
        fig.suptitle("Forecast Error Variance Decomposition of Inflation\n"
                     "(share explained by monetary policy shock)",
                     fontweight="bold", fontsize=13, y=1.05)

        # Full sample FEVD
        full_result = svar_results["full"]["result"]
        full_fevd = full_result.fevd(48).decomp  # shape: (n_vars, periods, n_vars)

        ax = axes[0]
        pi_idx = var_cols.index("inflation")
        n_periods = full_fevd.shape[1]
        horizons = np.arange(n_periods)
        for j, var_name in enumerate(var_cols):
            ax.plot(horizons, full_fevd[pi_idx, :, j], lw=2,
                    label=f"{var_name} shock")
        ax.set_xlabel("Months"); ax.set_ylabel("Share of variance")
        ax.set_title("Full sample", fontweight="bold")
        ax.legend(frameon=True, fontsize=8)
        ax.set_ylim(0, 1)

        for ax_idx, r in enumerate(sorted(regime_data.keys())):
            ax = axes[ax_idx + 1]
            r_result = regime_data[r]["result"]
            r_fevd = r_result.fevd(48).decomp
            n_periods_r = r_fevd.shape[1]
            horizons = np.arange(n_periods_r)
            for j, var_name in enumerate(var_cols):
                ax.plot(horizons, r_fevd[pi_idx, :, j], lw=2,
                        label=f"{var_name} shock")
            ax.set_xlabel("Months"); ax.set_ylabel("Share of variance")
            ax.set_title(regime_names[r], fontweight="bold")
            ax.legend(frameon=True, fontsize=8)
            ax.set_ylim(0, 1)

        fig.tight_layout()
        path3 = f"{output_path}/svar_fevd_comparison.png"
        fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path3}")
    else:
        path3 = None

    return [p for p in [path1, path2, path3] if p]


# ── Main ──
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(DATA_PATH)
    df = engineer_features(data)

    print("Fitting regimes...")
    regime_df, regime_names, n_regimes = get_regimes(df, n_regimes=2)
    print(f"  {n_regimes} regimes: {regime_names}")

    print("Running regime-dependent SVAR analysis...")
    svar_results = regime_svar_analysis(regime_df, regime_names, n_regimes)

    print("Plotting...")
    paths = plot_svar_results(svar_results, OUTPUT_PATH)

    print(f"\nOutputs: {paths}")
    print("Done.")

Loading data...
Merged: 857 obs (1954-07 to 2025-12)
Fitting regimes...
  2 regimes: {0: 'Low inflation (2.4%)', 1: 'High inflation (7.4%)'}
Running regime-dependent SVAR analysis...

REGIME-DEPENDENT STRUCTURAL VAR
  Ordering: inflation -> unemp_gap -> fed_funds (Cholesky)

  --- Full sample ---
  [Full sample] N=839, lags=6, AIC=-5.5, BIC=-5.2
    Computing bootstrap confidence intervals...
    Bootstrap: 500/500 successful replications

  --- Low inflation (2.4%) ---
  [Low inflation (2.4%)] N=628, lags=6, AIC=-5.6, BIC=-5.2
    Computing bootstrap confidence intervals...
    Bootstrap: 500/500 successful replications

  --- High inflation (7.4%) ---
  [High inflation (7.4%)] N=205, lags=6, AIC=-4.0, BIC=-3.0
    Computing bootstrap confidence intervals...
    Bootstrap: 500/500 successful replications

  === Monetary policy shock -> Inflation (cumulative) ===
   Horizon        Full  Low inflation (  High inflation 
  --------------------------------------------------
         1    